In [218]:
import pandas as pd
import seaborn as sns
import numpy as np
import linearmodels as lm
import matplotlib
import math
import statsmodels.formula.api as smf
import statsmodels.api as sm

from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import FirstDifferenceOLS
from linearmodels.panel import compare
from matplotlib import pyplot as plt

from pandas.api.types import is_numeric_dtype

pd.set_option('display.max_columns', 500)

%matplotlib inline

In [219]:
df=pd.read_csv('final_estimation_file_apta_added.csv',
                    encoding='utf-8')
df.drop(columns=['Unnamed: 0'],inplace=True)

In [221]:
tsd_data=pd.read_csv('tsd_pop_employment.csv',
                    encoding='utf-8')
tsd_data.drop(columns=['Unnamed: 0','CBSA Title'],inplace=True)
tsd_data.head()

,CBSA Code,STATE,Year,TOTAL_JOBS_WA_TRACTS,JOBS_LESS_THAN_1250_WA_TRACTS,JOBS_1250_3333_WA_TRACTS,JOBS_MORE_THAN_3333_WA_TRACTS,JOBS_WHITE_ALONE_WA_TRACTS,JOBS_BLACK_ALONE_WA_TRACTS,JOBS_HISPANIC_LATINO_WA_TRACTS,JOBS_LESS_THAN_40K_WA_TRACTS,JOBS_NON_WHITE_WA_TRACTS,JOBS_UPTO_HIGH_SCHOOL_WA_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TRACTS,TOTAL_JOBS_WA_TSD_TRACTS,JOBS_LESS_THAN_1250_WA_TSD_TRACTS,JOBS_1250_3333_WA_TSD_TRACTS,JOBS_MORE_THAN_3333_WA_TSD_TRACTS,JOBS_WHITE_ALONE_WA_TSD_TRACTS,JOBS_BLACK_ALONE_WA_TSD_TRACTS,JOBS_HISPANIC_LATINO_WA_TSD_TRACTS,JOBS_LESS_THAN_40K_WA_TSD_TRACTS,JOBS_NON_WHITE_WA_TSD_TRACTS,JOBS_UPTO_HIGH_SCHHOL_WA_TSD_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TSD_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS
0,10180,Texas,2002,57166.0,22735.0,25452.0,8979.0,0.0,0.0,0.0,48187.0,57166.0,0.0,0.0,9489.0,4531.0,3582.0,1376.0,0.0,0.0,0.0,8113.0,9489.0,0.0,0.0,58397.0,0.0,0.0,0.0,23086.0,25800.0,9511.0,48886.0,58397.0,0.0,0.0,1216.0,2355.0,0.0,0.0,0.0,954.0,1065.0,336.0,2019.0,2355.0,0.0,0.0
1,10180,Texas,2003,56917.0,21970.0,25046.0,9901.0,0.0,0.0,0.0,47016.0,56917.0,0.0,0.0,9484.0,4402.0,3471.0,1611.0,0.0,0.0,0.0,7873.0,9484.0,0.0,0.0,57221.0,0.0,0.0,0.0,22049.0,25074.0,10098.0,47123.0,57221.0,0.0,0.0,1216.0,2462.0,0.0,0.0,0.0,1032.0,1069.0,361.0,2101.0,2462.0,0.0,0.0
2,10180,Texas,2004,59025.0,22123.0,26375.0,10527.0,0.0,0.0,0.0,48498.0,59025.0,0.0,0.0,9599.0,4443.0,3594.0,1562.0,0.0,0.0,0.0,8037.0,9599.0,0.0,0.0,61419.0,0.0,0.0,0.0,23062.0,27199.0,11158.0,50261.0,61419.0,0.0,0.0,1216.0,2970.0,0.0,0.0,0.0,1231.0,1315.0,424.0,2546.0,2970.0,0.0,0.0
3,10180,Texas,2005,57879.0,21202.0,25353.0,11324.0,0.0,0.0,0.0,46555.0,57879.0,0.0,0.0,9645.0,4539.0,3500.0,1606.0,0.0,0.0,0.0,8039.0,9645.0,0.0,0.0,60001.0,0.0,0.0,0.0,21854.0,26081.0,12066.0,47935.0,60001.0,0.0,0.0,1216.0,2938.0,0.0,0.0,0.0,1134.0,1326.0,478.0,2460.0,2938.0,0.0,0.0
4,10180,Texas,2006,59588.0,21170.0,25759.0,12659.0,0.0,0.0,0.0,46929.0,59588.0,0.0,0.0,9819.0,4475.0,3516.0,1828.0,0.0,0.0,0.0,7991.0,9819.0,0.0,0.0,62311.0,0.0,0.0,0.0,22008.0,26639.0,13664.0,48647.0,62311.0,0.0,0.0,1216.0,2983.0,0.0,0.0,0.0,1122.0,1342.0,519.0,2464.0,2983.0,0.0,0.0


In [222]:
df=pd.merge(df,tsd_data,left_on=['CBSA','Year'],right_on=['CBSA Code','Year'],how='left')
df=df.drop_duplicates()
df.head()

,CBSA,MNAME,Mode,Year,UPT_ADJUSTED,VRM_ADJUSTED,FARE_ADJUSTED,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,AVG_FARE_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOTAL_JOBS_WA_TRACTS,JOBS_LESS_THAN_1250_WA_TRACTS,JOBS_1250_3333_WA_TRACTS,JOBS_MORE_THAN_3333_WA_TRACTS,JOBS_WHITE_ALONE_WA_TRACTS,JOBS_BLACK_ALONE_WA_TRACTS,JOBS_HISPANIC_LATINO_WA_TRACTS,JOBS_LESS_THAN_40K_WA_TRACTS,JOBS_NON_WHITE_WA_TRACTS,JOBS_UPTO_HIGH_SCHOOL_WA_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TRACTS,TOTAL_JOBS_WA_TSD_TRACTS,JOBS_LESS_THAN_1250_WA_TSD_TRACTS,JOBS_1250_3333_WA_TSD_TRACTS,JOBS_MORE_THAN_3333_WA_TSD_TRACTS,JOBS_WHITE_ALONE_WA_TSD_TRACTS,JOBS_BLACK_ALONE_WA_TSD_TRACTS,JOBS_HISPANIC_LATINO_WA_TSD_TRACTS,JOBS_LESS_THAN_40K_WA_TSD_TRACTS,JOBS_NON_WHITE_WA_TSD_TRACTS,JOBS_UPTO_HIGH_SCHHOL_WA_TSD_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TSD_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS
0,10420,"Akron, OH Metro Area",Bus,2002,5226385.660,4026372.388,5.965827e+05,2002,2018,0.114148,NaN,NaN,NaN,646583.7344,496141.0469,137193.6719,-389.53125,25405.20313,26028.32813,26651.07813,7117.03125,682354.11,7.55,8.94,83.85,24.64,62.9,12.46,265452.20,22830.14,58254.52,117083.25,46891.53,20392.77,8.60,43046.82813,55300.5625,39.49,46.56,13.95,21948.42,346147.17,368095.58,5.96,94.04,1.36,NaN,119786.0,14.88,1.40,2014,http://www.crainscleveland.com/article/2014082...,0,0,4004892.28,35567.28438,36439.65938,37311.50938,60265.55938,77420.7875,1.9040,0,0,0,NaN,NaN,0.0,NaN,0,0,0,1,1,H,24,NaN,309488.2031,87.4,6.2,0.8,1.7,0.1,0.7,3.1,Goes to negative when extrapolating. Possible ...,0.159808,2417.239652,both,24,10420.0,Ohio,314749.0,96006.0,131687.0,87056.0,0.0,0.0,0.0,227693.0,314749.0,0.0,0.0,81463.0,23319.0,33326.0,24818.0,0.0,0.0,0.0,56645.0,81463.0,0.0,0.0,305401.0,0.0,0.0,0.0,92556.0,127165.0,85680.0,219721.0,305401.0,0.0,0.0,13619.2,54517.0,0.0,0.0,0.0,18228.0,25397.0,10892.0,43625.0,54517.0,0.0,0.0
1,10420,"Akron, OH Metro Area",Bus,2003,5395901.888,3976568.624,2.318587e+06,2002,2018,0.429694,NaN,NaN,NaN,659397.0625,504631.8125,137582.3125,753.12500,25120.18750,25632.68750,25995.68750,10350.875,682850.56,8.81,8.66,82.79,24.33,63.0,12.68,269144.19,22073.44,67400.94,115443.00,44874.88,19351.94,8.20,43566.6

In [223]:
list(tsd_data)

['CBSA Code',
 'STATE',
 'Year',
 'TOTAL_JOBS_WA_TRACTS',
 'JOBS_LESS_THAN_1250_WA_TRACTS',
 'JOBS_1250_3333_WA_TRACTS',
 'JOBS_MORE_THAN_3333_WA_TRACTS',
 'JOBS_WHITE_ALONE_WA_TRACTS',
 'JOBS_BLACK_ALONE_WA_TRACTS',
 'JOBS_HISPANIC_LATINO_WA_TRACTS',
 'JOBS_LESS_THAN_40K_WA_TRACTS',
 'JOBS_NON_WHITE_WA_TRACTS',
 'JOBS_UPTO_HIGH_SCHOOL_WA_TRACTS',
 'JOBS_COLLEGE_OR_HIGHER_WA_TRACTS',
 'TOTAL_JOBS_WA_TSD_TRACTS',
 'JOBS_LESS_THAN_1250_WA_TSD_TRACTS',
 'JOBS_1250_3333_WA_TSD_TRACTS',
 'JOBS_MORE_THAN_3333_WA_TSD_TRACTS',
 'JOBS_WHITE_ALONE_WA_TSD_TRACTS',
 'JOBS_BLACK_ALONE_WA_TSD_TRACTS',
 'JOBS_HISPANIC_LATINO_WA_TSD_TRACTS',
 'JOBS_LESS_THAN_40K_WA_TSD_TRACTS',
 'JOBS_NON_WHITE_WA_TSD_TRACTS',
 'JOBS_UPTO_HIGH_SCHHOL_WA_TSD_TRACTS',
 'JOBS_COLLEGE_OR_HIGHER_WA_TSD_TRACTS',
 'TOTAL_JOBS_RA_TRACTS',
 'TOTAL_JOBS_WHITE_ALONE_RA_TRACTS',
 'TOTAL_JOBS_BLACK_ALONE_RA_TRACTS',
 'TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS',
 'TOTAL_JOBS_UNDER_1250_RA_TRACTS',
 'TOTAL_JOBS_BETN_1250_3333_RA_TRACTS',

In [226]:
# WAC data derived columns. To account for people coming into the TSD from outlying areas.
df['TSD_WA_EMP_PCT_LEHD']=100*df['TOTAL_JOBS_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TRACTS']
df['TSD_WA_EMP_PCT_CBSA']=100*df['TOTAL_JOBS_WA_TSD_TRACTS']/df['TOT_EMP_MSA']

df['JOBS_LESS_THAN_3333_WA_TRACTS']=df['JOBS_LESS_THAN_1250_WA_TRACTS']+df['JOBS_1250_3333_WA_TRACTS']
df['JOBS_LESS_THAN_3333_WA_TSD_TRACTS']=df['JOBS_LESS_THAN_1250_WA_TSD_TRACTS']+df['JOBS_1250_3333_WA_TSD_TRACTS']

df['TSD_WA_EMP_WAGE_UNDER_1250']=100*df['JOBS_LESS_THAN_1250_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TRACTS']
df['TSD_WA_EMP_WAGE_BETN_1250_3333']=100*df['JOBS_1250_3333_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TRACTS']
df['TSD_WA_EMP_WAGE_UNDER_3333']=100*df['JOBS_LESS_THAN_3333_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TRACTS']

df['TSD_WA_EMP_WAGE_UNDER_1250_PCT_WA']=100*df['JOBS_LESS_THAN_1250_WA_TSD_TRACTS']/df['JOBS_LESS_THAN_1250_WA_TRACTS']
df['TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_WA']=100*df['JOBS_1250_3333_WA_TSD_TRACTS']/df['JOBS_1250_3333_WA_TRACTS']
df['TSD_WA_EMP_WAGE_UNDER_3333_PCT_WA']=100*df['JOBS_LESS_THAN_3333_WA_TSD_TRACTS']/df['JOBS_LESS_THAN_3333_WA_TRACTS']

df['TSD_WA_EMP_WAGE_UNDER_1250_PCT_TSD']=100*df['JOBS_LESS_THAN_1250_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TSD_TRACTS']
df['TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_TSD']=100*df['JOBS_1250_3333_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TSD_TRACTS']
df['TSD_WA_EMP_WAGE_UNDER_3333_PCT_TSD']=100*df['JOBS_LESS_THAN_3333_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TSD_TRACTS']

#RAC data derived columns.

df['TSD_POP_PCT_LEHD']=100*df['TOTAL_JOBS_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS'] #RAC data
df['TSD_POP_PCT_CBSA']=100*df['TOTAL_JOBS_RA_TSD_TRACTS']/df['Tot_Pop']
df['TSD_POP_WAGE_UNDER_1250']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']
df['TSD_POP_WAGE_BETN_1250_3333']=100*df['TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']
df['TSD_POP_WAGE_UNDER_3333']=100*df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']

df['TSD_POP_WAGE_UNDER_1250_PCT_RA']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_UNDER_1250_RA_TRACTS']
df['TSD_POP_WAGE_BETN_1250_3333_PCT_RA']=100*df['TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_BETN_1250_3333_RA_TRACTS']
df['TSD_POP_WAGE_UNDER_3333_PCT_RA']=100*df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_UNDER_3333_RA_TRACTS']

df['TSD_POP_WAGE_UNDER_1250_PCT_TSD']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TSD_TRACTS']
df['TSD_POP_WAGE_BETN_1250_3333_PCT_TSD']=100*df['TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TSD_TRACTS']
df['TSD_POP_WAGE_UNDER_3333_PCT_TSD']=100*df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TSD_TRACTS']


In [227]:
# read updated TNC revenue data
tnc_rev=pd.read_csv('TNC_REVENUE_3.csv')
tnc_rev.drop(columns=['Unnamed: 0'],inplace=True)
tnc_rev=tnc_rev.drop_duplicates(['CBSA','MNAME','Year'])
tnc_rev.head()

,CBSA,MNAME,Year,TNC_TRIPS,TNC_BOOKINGS,TNC_REVENUE
0,10420,"Akron, OH Metro Area",2014,309683.000,4.656730e+06,4.790846e+05
1,10420,"Akron, OH Metro Area",2015,514097.390,7.703640e+06,1.762137e+06
2,10420,"Akron, OH Metro Area",2016,919909.510,1.343587e+07,3.011892e+06
3,10420,"Akron, OH Metro Area",2017,1428115.150,2.060033e+07,7.909986e+06
4,10420,"Akron, OH Metro Area",2018,2020372.375,2.940156e+07,1.114941e+07


In [228]:
df=pd.merge(df,tnc_rev,on=['CBSA','MNAME','Year'],how='left')
df.head()

,CBSA,MNAME,Mode,Year,UPT_ADJUSTED,VRM_ADJUSTED,FARE_ADJUSTED,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,AVG_FARE_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOTAL_JOBS_WA_TRACTS,JOBS_LESS_THAN_1250_WA_TRACTS,JOBS_1250_3333_WA_TRACTS,JOBS_MORE_THAN_3333_WA_TRACTS,JOBS_WHITE_ALONE_WA_TRACTS,JOBS_BLACK_ALONE_WA_TRACTS,JOBS_HISPANIC_LATINO_WA_TRACTS,JOBS_LESS_THAN_40K_WA_TRACTS,JOBS_NON_WHITE_WA_TRACTS,JOBS_UPTO_HIGH_SCHOOL_WA_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TRACTS,TOTAL_JOBS_WA_TSD_TRACTS,JOBS_LESS_THAN_1250_WA_TSD_TRACTS,JOBS_1250_3333_WA_TSD_TRACTS,JOBS_MORE_THAN_3333_WA_TSD_TRACTS,JOBS_WHITE_ALONE_WA_TSD_TRACTS,JOBS_BLACK_ALONE_WA_TSD_TRACTS,JOBS_HISPANIC_LATINO_WA_TSD_TRACTS,JOBS_LESS_THAN_40K_WA_TSD_TRACTS,JOBS_NON_WHITE_WA_TSD_TRACTS,JOBS_UPTO_HIGH_SCHHOL_WA_TSD_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TSD_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS,TSD_WA_EMP_PCT_LEHD,TSD_WA_EMP_PCT_CBSA,JOBS_LESS_THAN_3333_WA_TRACTS,JOBS_LESS_THAN_3333_WA_TSD_TRACTS,TSD_WA_EMP_WAGE_UNDER_1250,TSD_WA_EMP_WAGE_BETN_1250_3333,TSD_WA_EMP_WAGE_UNDER_3333,TSD_WA_EMP_WAGE_UNDER_1250_PCT_WA,TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_WA,TSD_WA_EMP_WAGE_UNDER_3333_PCT_WA,TSD_WA_EMP_WAGE_UNDER_1250_PCT_TSD,TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_TSD,TSD_WA_EMP_WAGE_UNDER_3333_PCT_TSD,TSD_POP_PCT_LEHD,TSD_POP_PCT_CBSA,TSD_POP_WAGE_UNDER_1250,TSD_POP_WAGE_BETN_1250_3333,TSD_POP_WAGE_UNDER_3333,TSD_POP_WAGE_UNDER_1250_PCT_RA,TSD_POP_WAGE_BETN_1250_3333_PCT_RA,TSD_POP_WAGE_UNDER_3333_PCT_RA,TSD_POP_WAGE_UNDER_1250_PCT_TSD,TSD_POP_WAGE_BETN_1250_3333_PCT_TSD,TSD_POP_WAGE_UNDER_3333_PCT_TSD,TNC_TRIPS,TNC_BOOKINGS,TNC_REVENUE
0,10420,"Akron, OH Metro Area",Bus,2002,5226385.660,4026372.388,5.965827e+05,2002,2018,0.114148,NaN,NaN,NaN,646583.7344,496141.0469,137193.6719,-389.53125,25405.20313,26028.32813,26651.07813,7117.03125,682354.11,7.55,8.94,83.85,24.64,62.9,12.46,265452.20,22830.14,58254.52,117083.25,46891.53,20392.77,8.60,43046.82813,55300.5625,39.49,46.56,13.95,21948.42,346147.17,368095.58,5.96,94.04,1.36,NaN,119786.0,14.88,1.40,2014,http://www.crainscleveland.com/article/2014082...,0,0,4004892.28,35567.28438,36439.65938,37311.50938,60265.55938,77420.7875,1.9040,0,0,0,NaN,NaN,0.0,NaN,0,0,0

In [229]:
#df=pd.merge(df,tnc_rev,on=['CBSA','MNAME','Year'],how='left')
df['TNC_REVENUE']=df['TNC_REVENUE'].fillna(0)
df['TNC_TRIPS']=df['TNC_TRIPS'].fillna(0)
df['TNC_BOOKINGS']=df['TNC_BOOKINGS'].fillna(0)
df.head()

,CBSA,MNAME,Mode,Year,UPT_ADJUSTED,VRM_ADJUSTED,FARE_ADJUSTED,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,AVG_FARE_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOTAL_JOBS_WA_TRACTS,JOBS_LESS_THAN_1250_WA_TRACTS,JOBS_1250_3333_WA_TRACTS,JOBS_MORE_THAN_3333_WA_TRACTS,JOBS_WHITE_ALONE_WA_TRACTS,JOBS_BLACK_ALONE_WA_TRACTS,JOBS_HISPANIC_LATINO_WA_TRACTS,JOBS_LESS_THAN_40K_WA_TRACTS,JOBS_NON_WHITE_WA_TRACTS,JOBS_UPTO_HIGH_SCHOOL_WA_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TRACTS,TOTAL_JOBS_WA_TSD_TRACTS,JOBS_LESS_THAN_1250_WA_TSD_TRACTS,JOBS_1250_3333_WA_TSD_TRACTS,JOBS_MORE_THAN_3333_WA_TSD_TRACTS,JOBS_WHITE_ALONE_WA_TSD_TRACTS,JOBS_BLACK_ALONE_WA_TSD_TRACTS,JOBS_HISPANIC_LATINO_WA_TSD_TRACTS,JOBS_LESS_THAN_40K_WA_TSD_TRACTS,JOBS_NON_WHITE_WA_TSD_TRACTS,JOBS_UPTO_HIGH_SCHHOL_WA_TSD_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TSD_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS,TSD_WA_EMP_PCT_LEHD,TSD_WA_EMP_PCT_CBSA,JOBS_LESS_THAN_3333_WA_TRACTS,JOBS_LESS_THAN_3333_WA_TSD_TRACTS,TSD_WA_EMP_WAGE_UNDER_1250,TSD_WA_EMP_WAGE_BETN_1250_3333,TSD_WA_EMP_WAGE_UNDER_3333,TSD_WA_EMP_WAGE_UNDER_1250_PCT_WA,TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_WA,TSD_WA_EMP_WAGE_UNDER_3333_PCT_WA,TSD_WA_EMP_WAGE_UNDER_1250_PCT_TSD,TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_TSD,TSD_WA_EMP_WAGE_UNDER_3333_PCT_TSD,TSD_POP_PCT_LEHD,TSD_POP_PCT_CBSA,TSD_POP_WAGE_UNDER_1250,TSD_POP_WAGE_BETN_1250_3333,TSD_POP_WAGE_UNDER_3333,TSD_POP_WAGE_UNDER_1250_PCT_RA,TSD_POP_WAGE_BETN_1250_3333_PCT_RA,TSD_POP_WAGE_UNDER_3333_PCT_RA,TSD_POP_WAGE_UNDER_1250_PCT_TSD,TSD_POP_WAGE_BETN_1250_3333_PCT_TSD,TSD_POP_WAGE_UNDER_3333_PCT_TSD,TNC_TRIPS,TNC_BOOKINGS,TNC_REVENUE
0,10420,"Akron, OH Metro Area",Bus,2002,5226385.660,4026372.388,5.965827e+05,2002,2018,0.114148,NaN,NaN,NaN,646583.7344,496141.0469,137193.6719,-389.53125,25405.20313,26028.32813,26651.07813,7117.03125,682354.11,7.55,8.94,83.85,24.64,62.9,12.46,265452.20,22830.14,58254.52,117083.25,46891.53,20392.77,8.60,43046.82813,55300.5625,39.49,46.56,13.95,21948.42,346147.17,368095.58,5.96,94.04,1.36,NaN,119786.0,14.88,1.40,2014,http://www.crainscleveland.com/article/2014082...,0,0,4004892.28,35567.28438,36439.65938,37311.50938,60265.55938,77420.7875,1.9040,0,0,0,NaN,NaN,0.0,NaN,0,0,0

In [230]:
# read in and join the maintenance and restructure data
mr=pd.read_csv('maintenance_restructure.csv',
                    encoding='utf-8')
mr = mr[['CBSA', 'Mode', 'Year', 'MAINTENANCE', 'MAINTENANCE_NYC', 'MAINTENANCE_WMATA', 'RESTRUCTURE']]

df = df.merge(mr, on=['CBSA', 'Mode', 'Year'], how='left')

In [231]:
# read in mechanical and other failures

failures=pd.read_csv('maintenance_final.csv',encoding='utf-8')

failures.drop(columns=['Unnamed: 0'],inplace=True)

df=df.merge(failures, on=['CBSA','MNAME','Mode','Year'],how='left')

In [232]:
df=df.rename(columns={'UPT_ADJUSTED':'UPT_ADJ','VRM_ADJUSTED':'VRM_ADJ','FARE_ADJUSTED':'FARE_ADJ','AVG_FARE_2018':'FARE_per_UPT_2018',
                     'Mecha_Failures':'Mechanical_Failures'})

In [233]:
df['Mechanical_Failures']=df['Mechanical_Failures'].fillna(0)
df['Total_Failures']=df['Total_Failures'].fillna(0)
df['MDBF_Mechanical']=np.where(df['Mechanical_Failures']>0,df['VRM_ADJ']/df['Mechanical_Failures'],df['VRM_ADJ'])
df['MDBF_Total']=np.where(df['Total_Failures']>0,df['VRM_ADJ']/df['Total_Failures'],df['VRM_ADJ'])

In [234]:
# read in and join the bike share updates
bsu=pd.read_csv('bike_share_update.csv',
                    encoding='utf-8')
bsu = bsu[['CBSA', 'Mode', 'Year', 'PBS_Flag_Update']]

df = df.merge(bsu, on=['CBSA', 'Mode', 'Year'], how='left')

#df['PBS_Flag_Missing'] = df['PBS_Flag'].apply(lambda x : x.isnan())
df['PBS_Flag'] = np.where(df['PBS_Flag'].isnull(), df['PBS_Flag_Update'], df['PBS_Flag'])

In [235]:
# 4 apta clusers
df['CLUSTER_APTA4'] = np.floor(df['CLUSTER_APTA']/10)

In [236]:
# exclude if ridership is zero--missing data
df = df[df['UPT_ADJ']>0]

In [237]:
# fill in missing data as needed

# min fare is zero
df['FARE_per_UPT_2018'] = df['FARE_per_UPT_2018'].apply(lambda x : max(x,0))

# max fare is $20
df['FARE_per_UPT_2018'] = df['FARE_per_UPT_2018'].apply(lambda x : min(x,20))

# zero vehicle HH can't go negative
df['HH_0Veh'] = df['HH_0Veh'].apply(lambda x : max(x,0))

In [238]:
df=df.drop_duplicates(['CBSA','Mode','Year'])

In [239]:
# exclude problematic data
old_len = len(df)
df = df[df['VRM_ADJ']>0]
new_len = len(df)
print('Excluding ' + str(old_len-new_len) + ' records with missing VRM') 

# exclude if problem flag 2 is set
old_len = len(df)
df = df[df['PROBLEM_FLAG2']!=1]
new_len = len(df)
print('Excluding ' + str(old_len-new_len) + ' records with problem flag set to 1') 

# exclude rail in low op-ex group
old_len = len(df)
df['LOW_OPEX'] = np.where(df['CLUSTER_APTA4']==3, 1, 0)
df['RAIL_FLAG'] = np.where(df['Mode']=='Rail', 1, 0)
df['LOW_OPEX_RAIL'] = df['LOW_OPEX'] * df['RAIL_FLAG']
df = df[df['LOW_OPEX_RAIL']==0]
new_len = len(df)
print('Excluding ' + str(old_len-new_len) + ' records with that have rail in the low op-ex group') 



Excluding 2 records with missing VRM
Excluding 36 records with problem flag set to 1
Excluding 48 records with that have rail in the low op-ex group


In [240]:
# exclude discontinuous data
df = df.sort_values(by=['CBSA', 'Mode', 'Year'])

excluded_records = 100
while excluded_records > 0: 
    old_len = len(df)
    df['YearDiff'] = df['Year'].shift(-1) - df['Year']
    df = df[(df['YearDiff']==1) | (df['Year']==2018)]
    excluded_records = old_len - len(df)
    print('Excluding ' + str(excluded_records) + ' records with discontinuities in year.') 




Excluding 1 records with discontinuities in year.
Excluding 1 records with discontinuities in year.
Excluding 0 records with discontinuities in year.


In [241]:
df['BUS_FLAG'] = np.where(df['Mode']=='Bus', 1, 0)
df['RAIL_FLAG'] = np.where(df['Mode']=='Rail', 1, 0)

In [242]:
# include population + employment
df['POP_EMP'] = df['Tot_Pop'] + df['TOT_EMP_MSA']
df['HH_EMP'] = df['Total_HH'] + df['TOT_EMP_MSA']


# percent of pop born outside USA
df['Tot_NonUSA_POP_pct'] = df['Tot_NonUSA_POP'] / df['Tot_Pop'] * 100

In [243]:
# time effects
df['YEARS_SINCE_2002']  = df['Year'] - 2002

df['YEARS_2002_2010']  = df['Year'].apply(lambda x : min(x-2002, 8))
df['YEARS_AFTER_2010'] = df['Year'].apply(lambda x : max(x-2010, 0))

df['YEARS_2002_2014']  = df['Year'].apply(lambda x : min(x-2002, 12))
df['YEARS_AFTER_2014'] = df['Year'].apply(lambda x : max(x-2014, 0))


df['YEARS_SINCE_2002_BUS']  = df['YEARS_SINCE_2002'] * df['BUS_FLAG']
df['YEARS_SINCE_2002_RAIL'] = df['YEARS_SINCE_2002'] * df['RAIL_FLAG']

df['YEARS_2002_2010_BUS']   = df['YEARS_2002_2010'] * df['BUS_FLAG']
df['YEARS_AFTER_2010_BUS']  = df['YEARS_AFTER_2010']* df['BUS_FLAG']
df['YEARS_2002_2010_RAIL']  = df['YEARS_2002_2010'] * df['RAIL_FLAG']
df['YEARS_AFTER_2010_RAIL'] = df['YEARS_AFTER_2010']* df['RAIL_FLAG']

df['YEARS_2002_2014_BUS']   = df['YEARS_2002_2014'] * df['BUS_FLAG']
df['YEARS_AFTER_2014_BUS']  = df['YEARS_AFTER_2014']* df['BUS_FLAG']
df['YEARS_2002_2014_RAIL']  = df['YEARS_2002_2014'] * df['RAIL_FLAG']
df['YEARS_AFTER_2014_RAIL'] = df['YEARS_AFTER_2014']* df['RAIL_FLAG']

In [244]:
# more time effects

df['YEARS_AFTER_2012'] = df['Year'].apply(lambda x : max(x-2012, 0))

df['TNC_YEARS_AFTER_2012'] = df['YEARS_AFTER_2012'] * df['TNC_FLAG']

df['TNC_YEARS_AFTER_2012_BUS'] = df['TNC_YEARS_AFTER_2012'] * df['BUS_FLAG']
df['TNC_YEARS_AFTER_2012_RAIL'] = df['TNC_YEARS_AFTER_2012'] * df['RAIL_FLAG']

In [245]:

df['YEARS_SINCE_TNC_2012'] = np.minimum(df['YEARS_SINCE_TNC'], df['YEARS_AFTER_2012'])

df['YEARS_SINCE_TNC_2012_BUS'] = df['YEARS_SINCE_TNC_2012'] * df['BUS_FLAG']
df['YEARS_SINCE_TNC_2012_RAIL'] = df['YEARS_SINCE_TNC_2012'] * df['RAIL_FLAG']

In [246]:
# test rail ramp up period
df['YEARS_SINCE_RAIL_START'] = (df['Year'] - df['min_year']) * df['RAIL_FLAG']
df['YEARS_SINCE_RAIL_START'] = np.where(df['min_year']==2002, 0, df['YEARS_SINCE_RAIL_START'])

df['YEARS_SINCE_RAIL_START_1'] = df['YEARS_SINCE_RAIL_START'].apply(lambda x : min(x, 1))
df['YEARS_SINCE_RAIL_START_2'] = df['YEARS_SINCE_RAIL_START'].apply(lambda x : min(x, 2))
df['YEARS_SINCE_RAIL_START_3'] = df['YEARS_SINCE_RAIL_START'].apply(lambda x : min(x, 3))



In [247]:
# calculate bus vs rail stuff

# various bus/rail specifics

df['TNC_FLAG_BUS'] = df['TNC_FLAG'] * df['BUS_FLAG']
df['TNC_FLAG_RAIL'] = df['TNC_FLAG'] * df['RAIL_FLAG']

df['YEARS_SINCE_TNC_BUS'] = df['YEARS_SINCE_TNC'] * df['BUS_FLAG']
df['YEARS_SINCE_TNC_RAIL'] = df['YEARS_SINCE_TNC'] * df['RAIL_FLAG']

df['dockless_flag'] = df['dockCt'].apply(lambda x : min(x,1))
df['scooter_flag']  = df['scooterCt'].apply(lambda x : min(x,1))

df['BIKE_SHARE'] = df['PBS_Flag'] + df['dockless_flag']
df['BIKE_SHARE'] = df['BIKE_SHARE'].apply(lambda x : min(x,1))


df['PBS_Flag_BUS']   = df['PBS_Flag']   * df['BUS_FLAG']
df['dockCt_BUS']     = df['dockCt']     * df['BUS_FLAG']
df['docklessCt_BUS'] = df['docklessCt'] * df['BUS_FLAG']
df['scooterCt_BUS']  = df['scooterCt']  * df['BUS_FLAG']
df['dockless_flag_BUS'] = df['dockless_flag'] * df['BUS_FLAG']
df['scooter_flag_BUS']  = df['scooter_flag']  * df['BUS_FLAG']
df['BIKE_SHARE_BUS']  = df['BIKE_SHARE']  * df['BUS_FLAG']


df['PBS_Flag_RAIL']   = df['PBS_Flag']   * df['RAIL_FLAG']
df['dockCt_RAIL']     = df['dockCt']     * df['RAIL_FLAG']
df['docklessCt_RAIL'] = df['docklessCt'] * df['RAIL_FLAG']
df['scooterCt_RAIL']  = df['scooterCt']  * df['RAIL_FLAG']
df['dockless_flag_RAIL'] = df['dockless_flag'] * df['RAIL_FLAG']
df['scooter_flag_RAIL']  = df['scooter_flag']  * df['RAIL_FLAG']
df['BIKE_SHARE_RAIL']  = df['BIKE_SHARE']  * df['RAIL_FLAG']

In [248]:
# convert JTW to numeric columns
df['JTW_DA_PCT']        = df['JTW_DA_PCT'].replace('#VALUE!', np.nan)
df['JTW_CARPOOLED_PCT'] = df['JTW_CARPOOLED_PCT'].replace('#VALUE!', np.nan)
df['JTW_TRANSIT_PCT']   = df['JTW_TRANSIT_PCT'].replace('#VALUE!', np.nan)
df['JTW_WALK_PCT']      = df['JTW_WALK_PCT'].replace('#VALUE!', np.nan)
df['JTW_BICYCLE_PCT']   = df['JTW_BICYCLE_PCT'].replace('#VALUE!', np.nan)
df['JTW_OTHER_PCT']     = df['JTW_OTHER_PCT'].replace('#VALUE!', np.nan)
df['JTW_HOME_PCT']      = df['JTW_HOME_PCT'].replace('#VALUE!', np.nan)


df['JTW_DA_PCT']        = df['JTW_DA_PCT'].replace('N', np.nan)
df['JTW_CARPOOLED_PCT'] = df['JTW_CARPOOLED_PCT'].replace('N', np.nan)
df['JTW_TRANSIT_PCT']   = df['JTW_TRANSIT_PCT'].replace('N', np.nan)
df['JTW_WALK_PCT']      = df['JTW_WALK_PCT'].replace('N', np.nan)
df['JTW_BICYCLE_PCT']   = df['JTW_BICYCLE_PCT'].replace('N', np.nan)
df['JTW_OTHER_PCT']     = df['JTW_OTHER_PCT'].replace('N', np.nan)
df['JTW_HOME_PCT']      = df['JTW_HOME_PCT'].replace('N', np.nan)



In [249]:
# convert JTW to numeric columns
df['JTW_DA_PCT']        = df['JTW_DA_PCT'].astype(float)
df['JTW_CARPOOLED_PCT'] = df['JTW_CARPOOLED_PCT'].astype(float)
df['JTW_TRANSIT_PCT']   = df['JTW_TRANSIT_PCT'].astype(float)
df['JTW_WALK_PCT']      = df['JTW_WALK_PCT'].astype(float)
df['JTW_BICYCLE_PCT']   = df['JTW_BICYCLE_PCT'].astype(float)
df['JTW_OTHER_PCT']     = df['JTW_OTHER_PCT'].astype(float)
df['JTW_HOME_PCT']      = df['JTW_HOME_PCT'].astype(float)

In [250]:
# immigrant population percentage
df['Tot_NonUSA_POP_pct'] = df['Tot_NonUSA_POP'] / df['Tot_Pop'] * 100

In [251]:
# bike share plus scooters for rail
df['BIKE_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL']

# different spec of bike and scooter
df['BIKE_SCOOTER_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] + df['scooter_flag_RAIL']

# different spec of bike and scooter
df['BIKE_SCOOTER_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] + df['scooter_flag_RAIL']

# merge bike and scooter for bus
df['BIKE_SCOOTER_SCOOTER_BUS'] = df['BIKE_SHARE_BUS'] + df['scooter_flag_BUS'] + df['scooter_flag_BUS']

# merge bike and scooter for bus
df['BIKE_SCOOTER_BUS'] = df['BIKE_SHARE_BUS'] + df['scooter_flag_BUS'] 
df['BIKE_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] 

# scooter or bike
df['BIKE_OR_SCOOTER_BUS']  = df['BIKE_SHARE_BUS'] + df['scooter_flag_BUS'] 
df['BIKE_OR_SCOOTER_BUS']  = df['BIKE_OR_SCOOTER_BUS'].apply(lambda x : min(x, 1))

df['BIKE_OR_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] 
df['BIKE_OR_SCOOTER_RAIL'] = df['BIKE_OR_SCOOTER_RAIL'].apply(lambda x : min(x, 1))

# calculate average vehicles
df['AVG_VEHS'] = (df['HH_1Veh'] + 2*df['HH_2Veh'] + 3*df['HH_3Veh'] + 4*df['HH_4+Veh']) / (df['HH_0Veh'] + df['HH_1Veh'] + df['HH_2Veh'] + df['HH_3Veh'] + df['HH_4+Veh'])
df['AVG_VEHS_log'] = np.log(1+df['AVG_VEHS'])

In [252]:
# walk bike other commuting
df['JTW_WALK_BIKE_OTHER_PCT'] = df['JTW_WALK_PCT'] + df['JTW_BICYCLE_PCT'] + df['JTW_OTHER_PCT']

# walk bike commuting
df['JTW_WALK_BIKE_PCT'] = df['JTW_WALK_PCT'] + df['JTW_BICYCLE_PCT']

In [253]:
df['VRM_ADJ_BUS'] = df['VRM_ADJ'] * df['BUS_FLAG']
df['VRM_ADJ_RAIL'] = df['VRM_ADJ'] * df['RAIL_FLAG']


#maintenance effect on competing mode
df['MDBF_Mechanical_RAIL']  = df['MDBF_Mechanical'] * df['RAIL_FLAG']
df['MDBF_Mechanical_BUS'] = df['MDBF_Mechanical']  * df['BUS_FLAG']

df['MDBF_Total_RAIL']  = df['MDBF_Total'] * df['RAIL_FLAG']
df['MDBF_Total_BUS'] = df['MDBF_Total']  * df['BUS_FLAG']

In [254]:
# segment TNCs by cluster
df['HI_OPEX']  = np.where(df['CLUSTER_APTA4']==1, 1, 0)
df['MID_OPEX'] = np.where(df['CLUSTER_APTA4']==2, 1, 0)
df['LOW_OPEX'] = np.where(df['CLUSTER_APTA4']==3, 1, 0)
df['NEW_YORK'] = np.where(df['CLUSTER_APTA4']==10, 1, 0)

df['YEARS_SINCE_TNC_BUS_HI']  = df['YEARS_SINCE_TNC_BUS'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_BUS_MID'] = df['YEARS_SINCE_TNC_BUS'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_BUS_LOW'] = df['YEARS_SINCE_TNC_BUS'] * df['LOW_OPEX']
df['YEARS_SINCE_TNC_BUS_NY']  = df['YEARS_SINCE_TNC_BUS'] * df['NEW_YORK']

df['YEARS_SINCE_TNC_RAIL_HI']  = df['YEARS_SINCE_TNC_RAIL'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_RAIL_MID'] = df['YEARS_SINCE_TNC_RAIL'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_RAIL_LOW'] = df['YEARS_SINCE_TNC_RAIL'] * df['LOW_OPEX']
df['YEARS_SINCE_TNC_RAIL_NY']  = df['YEARS_SINCE_TNC_RAIL'] * df['NEW_YORK']

df['YEARS_SINCE_TNC_BUS_HINY']  = df['YEARS_SINCE_TNC_BUS_HI'] + df['YEARS_SINCE_TNC_BUS_NY']
df['YEARS_SINCE_TNC_RAIL_HINY']  = df['YEARS_SINCE_TNC_RAIL_HI'] + df['YEARS_SINCE_TNC_RAIL_NY']

df['YEARS_SINCE_TNC_BUS_MIDLOW']  = df['YEARS_SINCE_TNC_BUS_MID'] + df['YEARS_SINCE_TNC_BUS_LOW']
df['YEARS_SINCE_TNC_RAIL_MIDLOW']  = df['YEARS_SINCE_TNC_RAIL_MID'] + df['YEARS_SINCE_TNC_RAIL_LOW']

In [255]:
# bikes and scooters by segment

df['BIKE_SHARE_HI']  = df['BIKE_SHARE'] * df['HI_OPEX']
df['BIKE_SHARE_MID'] = df['BIKE_SHARE'] * df['MID_OPEX']
df['BIKE_SHARE_LOW'] = df['BIKE_SHARE'] * df['LOW_OPEX']
df['BIKE_SHARE_NY']  = df['BIKE_SHARE'] * df['NEW_YORK']

df['scooter_flag_HI']  = df['scooter_flag'] * df['HI_OPEX']
df['scooter_flag_MID'] = df['scooter_flag'] * df['MID_OPEX']
df['scooter_flag_LOW'] = df['scooter_flag'] * df['LOW_OPEX']
df['scooter_flag_NY']  = df['scooter_flag'] * df['NEW_YORK']


df['BIKE_SHARE_HINY']  = df['BIKE_SHARE_HI'] + df['BIKE_SHARE_NY']
df['BIKE_SHARE_MIDLOW']  = df['BIKE_SHARE_MID'] + df['BIKE_SHARE_LOW']

df['scooter_flag_HINY']  = df['scooter_flag_HI'] + df['scooter_flag_NY']
df['scooter_flag_MIDLOW']  = df['scooter_flag_MID'] + df['scooter_flag_LOW']

In [256]:
# tnc_rev=pd.read_csv('tnc_revenue.csv',encoding='utf-8')
# tnc_rev.head()

In [58]:
# tnc_rev.drop(columns=['Unnamed: 0'],inplace=True)

In [59]:
# df=pd.merge(df,tnc_rev,on=['CBSA',
#  'MNAME',
#  'Mode',
#  'Year',
#  'TNC_FLAG',
#  'YEARS_SINCE_TNC'],how='left')
# df.head()

In [60]:
# df['UBER_NET_REVENUE']=df['UBER_NET_REVENUE'].fillna(0)
# df['UBER_REV_SCALED']=df['UBER_REV_SCALED'].fillna(0)
# df.head(20)

In [37]:
# df['UBER_REV_SCALED_BUS'] = df['UBER_REV_SCALED'] * df['BUS_FLAG']
# df['UBER_REV_SCALED_RAIL'] = df['UBER_REV_SCALED'] * df['RAIL_FLAG']

# df['UBER_REV_SCALED_BUS_HI']  = df['UBER_REV_SCALED_BUS'] * df['HI_OPEX']
# df['UBER_REV_SCALED_BUS_MID'] = df['UBER_REV_SCALED_BUS'] * df['MID_OPEX']
# df['UBER_REV_SCALED_BUS_LOW'] = df['UBER_REV_SCALED_BUS'] * df['LOW_OPEX']
# df['UBER_REV_SCALED_BUS_NY']  = df['UBER_REV_SCALED_BUS'] * df['NEW_YORK']

# df['UBER_REV_SCALED_RAIL_HI']  = df['UBER_REV_SCALED_RAIL'] * df['HI_OPEX']
# df['UBER_REV_SCALED_RAIL_MID'] = df['UBER_REV_SCALED_RAIL'] * df['MID_OPEX']
# df['UBER_REV_SCALED_RAIL_LOW'] = df['UBER_REV_SCALED_RAIL'] * df['LOW_OPEX']
# df['UBER_REV_SCALED_RAIL_NY']  = df['UBER_REV_SCALED_RAIL'] * df['NEW_YORK']

# df['UBER_REV_SCALED_BUS_HINY']  = df['UBER_REV_SCALED_BUS_HI'] + df['UBER_REV_SCALED_BUS_NY']
# df['UBER_REV_SCALED_RAIL_HINY']  = df['UBER_REV_SCALED_RAIL_HI'] + df['UBER_REV_SCALED_RAIL_NY']

# df['UBER_REV_SCALED_BUS_MIDLOW']  = df['UBER_REV_SCALED_BUS_MID'] + df['UBER_REV_SCALED_BUS_LOW']
# df['UBER_REV_SCALED_RAIL_MIDLOW']  = df['UBER_REV_SCALED_RAIL_MID'] + df['UBER_REV_SCALED_RAIL_LOW']

In [38]:
# df['UBER_NET_REVENUE_BUS'] = df['UBER_NET_REVENUE'] * df['BUS_FLAG']
# df['UBER_NET_REVENUE_RAIL'] = df['UBER_NET_REVENUE'] * df['RAIL_FLAG']

# df['UBER_NET_REVENUE_BUS_HI']  = df['UBER_NET_REVENUE_BUS'] * df['HI_OPEX']
# df['UBER_NET_REVENUE_BUS_MID'] = df['UBER_NET_REVENUE_BUS'] * df['MID_OPEX']
# df['UBER_NET_REVENUE_BUS_LOW'] = df['UBER_NET_REVENUE_BUS'] * df['LOW_OPEX']
# df['UBER_NET_REVENUE_BUS_NY']  = df['UBER_NET_REVENUE_BUS'] * df['NEW_YORK']

# df['UBER_NET_REVENUE_RAIL_HI']  = df['UBER_NET_REVENUE_RAIL'] * df['HI_OPEX']
# df['UBER_NET_REVENUE_RAIL_MID'] = df['UBER_NET_REVENUE_RAIL'] * df['MID_OPEX']
# df['UBER_NET_REVENUE_RAIL_LOW'] = df['UBER_NET_REVENUE_RAIL'] * df['LOW_OPEX']
# df['UBER_NET_REVENUE_RAIL_NY']  = df['UBER_NET_REVENUE_RAIL'] * df['NEW_YORK']

# df['UBER_NET_REVENUE_BUS_HINY']  = df['UBER_NET_REVENUE_BUS_HI'] + df['UBER_NET_REVENUE_BUS_NY']
# df['UBER_NET_REVENUE_RAIL_HINY']  = df['UBER_NET_REVENUE_RAIL_HI'] + df['UBER_NET_REVENUE_RAIL_NY']

# df['UBER_NET_REVENUE_BUS_MIDLOW']  = df['UBER_NET_REVENUE_BUS_MID'] + df['UBER_NET_REVENUE_BUS_LOW']
# df['UBER_NET_REVENUE_RAIL_MIDLOW']  = df['UBER_NET_REVENUE_RAIL_MID'] + df['UBER_NET_REVENUE_RAIL_LOW']

In [173]:
list(df)

['CBSA',
 'MNAME',
 'Mode',
 'Year',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'NOTE',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Native_Outcountry_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'Link_x',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE

In [257]:
columns=['TNC_REVENUE','TNC_TRIPS','TNC_BOOKINGS']
dummy=['HI_OPEX','MID_OPEX','LOW_OPEX','NEW_YORK']

for col in columns:
    df[col+'_BUS']=df[col]*df['BUS_FLAG']
    df[col+'_RAIL']=df[col]*df['RAIL_FLAG']
    
    for name in dummy:
        df[col+'_BUS_'+name]  = df[col]*df['BUS_FLAG'] * df[name]
        df[col+'_RAIL_'+name]  = df[col]*df['RAIL_FLAG'] * df[name]
#         df[col+'_BUS_MID'] = df['UBER_NET_REVENUE_BUS'] * df['MID_OPEX']
#         df[col+'_BUS_LOW'] = df['UBER_NET_REVENUE_BUS'] * df['LOW_OPEX']
#         df[col+'_BUS_NY']  = df['UBER_NET_REVENUE_BUS'] * df['NEW_YORK']

#         df['TNC_REVENUE_RAIL_HI']  = df['UBER_NET_REVENUE_RAIL'] * df['HI_OPEX']
#         df['TNC_REVENUE_RAIL_MID'] = df['UBER_NET_REVENUE_RAIL'] * df['MID_OPEX']
#         df['TNC_REVENUE_RAIL_LOW'] = df['UBER_NET_REVENUE_RAIL'] * df['LOW_OPEX']
#         df['TNC_REVENUE_RAIL_NY']  = df['UBER_NET_REVENUE_RAIL'] * df['NEW_YORK']

    df[col+'_BUS_HINY']  = df[col+'_BUS_HI_OPEX'] + df[col+'_BUS_NEW_YORK']
    df[col+'_BUS_MIDLOW']  = df[col+'_BUS_MID_OPEX'] + df[col+'_BUS_LOW_OPEX']
    df[col+'_BUS_HINY']  = df[col+'_BUS_HI_OPEX'] + df[col+'_BUS_NEW_YORK']
    
    df[col+'_RAIL_HINY']  = df[col+'_RAIL_HI_OPEX'] + df[col+'_RAIL_NEW_YORK']
    df[col+'_RAIL_MIDLOW']  = df[col+'_RAIL_MID_OPEX'] + df[col+'_RAIL_LOW_OPEX']
    df[col+'_RAIL_HINY']  = df[col+'_RAIL_HI_OPEX'] + df[col+'_RAIL_NEW_YORK']

# df['TNC_REVENUE_BUS_HINY']  = df['TNC_REVENUE_BUS_HI'] + df['TNC_REVENUE_BUS_NY']
# df['TNC_REVENUE_RAIL_HINY']  = df['TNC_REVENUE_RAIL_HI'] + df['UBER_NET_REVENUE_RAIL_NY']

# df['TNC_REVENUE_BUS_MIDLOW']  = df['UBER_NET_REVENUE_BUS_MID'] + df['UBER_NET_REVENUE_BUS_LOW']
# df['TNC_REVENUE_RAIL_MIDLOW']  = df['UBER_NET_REVENUE_RAIL_MID'] + df['UBER_NET_REVENUE_RAIL_LOW']

In [258]:
# set the indices
df['ID'] = df['MNAME'] + '-' + df['Mode']
df=df.set_index(['ID','Year'])

In [259]:
# keep only the numeric columns -- the estimation will give an error otherwise
df = df.select_dtypes(include=[np.number])

In [260]:
# create a log of all fields
for col in df.columns:
    df[col+'_log'] = np.log(df[col]+1)

In [261]:
# Test TNC ramp up

df['YEARS_SINCE_TNC_BUS_2'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 2))
df['YEARS_SINCE_TNC_BUS_3'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 3))
df['YEARS_SINCE_TNC_BUS_4'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 4))
df['YEARS_SINCE_TNC_BUS_5'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 5))

df['YEARS_SINCE_TNC_RAIL_2'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 2))
df['YEARS_SINCE_TNC_RAIL_3'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 3))
df['YEARS_SINCE_TNC_RAIL_4'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 4))
df['YEARS_SINCE_TNC_RAIL_5'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 5))

In [262]:
# count the first year of TNC presence
df['YEARS_SINCE_TNC_BUS2'] = df['YEARS_SINCE_TNC_BUS'] + df['TNC_FLAG_BUS']
df['YEARS_SINCE_TNC_RAIL2'] = df['YEARS_SINCE_TNC_RAIL'] + df['TNC_FLAG_RAIL']


In [263]:
## start in first year TNCs show up

df['YEARS_SINCE_TNC_BUS2_NY']  = df['YEARS_SINCE_TNC_BUS2'] * df['NEW_YORK']
df['YEARS_SINCE_TNC_BUS2_HI']  = df['YEARS_SINCE_TNC_BUS2'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_BUS2_MID'] = df['YEARS_SINCE_TNC_BUS2'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_BUS2_LOW'] = df['YEARS_SINCE_TNC_BUS2'] * df['LOW_OPEX']

df['YEARS_SINCE_TNC_BUS2_HINY']   = df['YEARS_SINCE_TNC_BUS2_NY'] + df['YEARS_SINCE_TNC_BUS2_HI']
df['YEARS_SINCE_TNC_BUS2_MIDLOW'] = df['YEARS_SINCE_TNC_BUS2_MID'] + df['YEARS_SINCE_TNC_BUS2_LOW']

df['YEARS_SINCE_TNC_RAIL2_NY']  = df['YEARS_SINCE_TNC_RAIL2'] * df['NEW_YORK']
df['YEARS_SINCE_TNC_RAIL2_HI']  = df['YEARS_SINCE_TNC_RAIL2'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_RAIL2_MID'] = df['YEARS_SINCE_TNC_RAIL2'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_RAIL2_LOW'] = df['YEARS_SINCE_TNC_RAIL2'] * df['LOW_OPEX']

df['YEARS_SINCE_TNC_RAIL2_HINY']   = df['YEARS_SINCE_TNC_RAIL2_NY'] + df['YEARS_SINCE_TNC_RAIL2_HI']
df['YEARS_SINCE_TNC_RAIL2_MIDLOW'] = df['YEARS_SINCE_TNC_RAIL2_MID'] + df['YEARS_SINCE_TNC_RAIL2_LOW']

In [264]:
## BIKE SHARE SEGMENTATION

df['BIKE_SHARE_NY']  = df['BIKE_SHARE'] * df['NEW_YORK']
df['BIKE_SHARE_HI']  = df['BIKE_SHARE'] * df['HI_OPEX']
df['BIKE_SHARE_MID'] = df['BIKE_SHARE'] * df['MID_OPEX']
df['BIKE_SHARE_LOW'] = df['BIKE_SHARE'] * df['LOW_OPEX']

df['BIKE_SHARE_HINY']   = df['BIKE_SHARE_NY'] + df['BIKE_SHARE_HI']
df['BIKE_SHARE_MIDLOW'] = df['BIKE_SHARE_MID'] + df['BIKE_SHARE_LOW']

df['scooter_flag_NY']  = df['scooter_flag'] * df['NEW_YORK']
df['scooter_flag_HI']  = df['scooter_flag'] * df['HI_OPEX']
df['scooter_flag_MID'] = df['scooter_flag'] * df['MID_OPEX']
df['scooter_flag_LOW'] = df['scooter_flag'] * df['LOW_OPEX']

df['scooter_flag_HINY']   = df['scooter_flag_NY'] + df['scooter_flag_HI']
df['scooter_flag_MIDLOW'] = df['scooter_flag_MID'] + df['scooter_flag_LOW']



## Model Estimation

In [296]:
# base model 1 from 9-5
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res2=mod.fit()
print(res2)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6364
Estimator:                   PanelOLS   R-squared (Between):              0.9404
No. Observations:                4114   R-squared (Within):               0.6364
Date:                Mon, May 04 2020   R-squared (Overall):              0.9408
Time:                        04:40:33   Log-likelihood                    1340.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      558.86
Entities:                         270   P-value                           0.0000
Avg Obs:                       15.237   Distribution:                 F(12,3832)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             558.86
                            

### Transit Supportive Density

In [280]:
# switch to LEHD population percent

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6448
Estimator:                   PanelOLS   R-squared (Between):              0.9350
No. Observations:                4002   R-squared (Within):               0.6448
Date:                Mon, May 04 2020   R-squared (Overall):              0.9355
Time:                        04:31:11   Log-likelihood                    1349.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      563.88
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             563.88
                            

In [301]:
# try with LEHD wac data

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6447
Estimator:                   PanelOLS   R-squared (Between):              0.9436
No. Observations:                4002   R-squared (Within):               0.6447
Date:                Mon, May 04 2020   R-squared (Overall):              0.9440
Time:                        04:49:32   Log-likelihood                    1349.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      563.64
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             563.64
                            

In [302]:
# try with LEHD wac and rac data

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_PCT_LEHD \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6451
Estimator:                   PanelOLS   R-squared (Between):              0.9421
No. Observations:                4002   R-squared (Within):               0.6451
Date:                Mon, May 04 2020   R-squared (Overall):              0.9425
Time:                        05:38:49   Log-likelihood                    1351.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      521.07
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(13,3727)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             521.07
                            

In [304]:
# add up RAC and WAC data. 
#RAC is the number of employed people living in a tract. the aggregation means the total number of people
#living in the TSD in an MSA
# WAC is the number of people working in a tract. the aggregation means that the total number of people employed
#in the TSD in an MSA
#Summing these two up, we get the total number of people living and working in the TSD of an MSA. 
#This value represents the intended transit customers in an MSA.

df['TSD_POP_EMP']=df['TOTAL_JOBS_WA_TSD_TRACTS']+df['TOTAL_JOBS_RA_TSD_TRACTS']
df['TSD_POP_EMP_TOTAL']=df['TOTAL_JOBS_WA_TRACTS']+df['TOTAL_JOBS_RA_TRACTS']
df['TSD_POP_EMP_PCT']=df['TSD_POP_EMP']/df['TSD_POP_EMP_TOTAL']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #reasonable coefficient on JTW_HOME_PCT
# use this one

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6451
Estimator:                   PanelOLS   R-squared (Between):              0.9431
No. Observations:                4002   R-squared (Within):               0.6451
Date:                Mon, May 04 2020   R-squared (Overall):              0.9435
Time:                        05:46:14   Log-likelihood                    1351.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      564.77
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             564.77
                            

In [306]:
#Testing the effect of the spread of low-wage population in the MSA

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

#Seems to be some kind of interaction between the two TSD variables.

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6457
Estimator:                   PanelOLS   R-squared (Between):              0.9496
No. Observations:                4002   R-squared (Within):               0.6457
Date:                Mon, May 04 2020   R-squared (Overall):              0.9500
Time:                        06:20:35   Log-likelihood                    1355.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      522.55
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(13,3727)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             522.55
                            

In [309]:
df['TSD_POP_EMP_WAGE_UNDER_1250']=df['JOBS_LESS_THAN_1250_WA_TSD_TRACTS']+df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']
df['TSD_POP_EMP_WAGE_UNDER_1250_PCT']=df['TSD_POP_EMP_WAGE_UNDER_1250']/df['TSD_POP_EMP_TOTAL']
df['TSD_POP_EMP_WAGE_UNDER_3333']=df['JOBS_LESS_THAN_3333_WA_TSD_TRACTS']+df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']
df['TSD_POP_EMP_WAGE_UNDER_3333_PCT']=df['TSD_POP_EMP_WAGE_UNDER_3333']/df['TSD_POP_EMP_TOTAL']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_WAGE_UNDER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6448
Estimator:                   PanelOLS   R-squared (Between):              0.9534
No. Observations:                4002   R-squared (Within):               0.6448
Date:                Mon, May 04 2020   R-squared (Overall):              0.9538
Time:                        06:48:08   Log-likelihood                    1349.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      563.84
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             563.84
                            

In [453]:
df['TSD_POP_EMP_WAGE_OVER_1250']=df['TSD_POP_EMP_TOTAL']-df['TSD_POP_EMP_WAGE_UNDER_1250']
df['TSD_POP_EMP_WAGE_OVER_1250_PCT']=df['TSD_POP_EMP_WAGE_OVER_1250']/df['TSD_POP_EMP_TOTAL']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_WAGE_UNDER_1250_PCT \
                    + TSD_POP_EMP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6448
Estimator:                   PanelOLS   R-squared (Between):              0.9098
No. Observations:                4002   R-squared (Within):               0.6448
Date:                Tue, May 05 2020   R-squared (Overall):              0.9055
Time:                        10:23:08   Log-likelihood                    1349.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      563.84
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          6.293e+04
                            

In [310]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_WAGE_UNDER_3333_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6449
Estimator:                   PanelOLS   R-squared (Between):              0.9527
No. Observations:                4002   R-squared (Within):               0.6449
Date:                Mon, May 04 2020   R-squared (Overall):              0.9530
Time:                        06:48:27   Log-likelihood                    1350.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      564.10
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             564.10
                            

In [454]:
df['TSD_POP_EMP_WAGE_OVER_3333']=df['TSD_POP_EMP_TOTAL']-df['TSD_POP_EMP_WAGE_UNDER_3333']
df['TSD_POP_EMP_WAGE_OVER_3333_PCT']=df['TSD_POP_EMP_WAGE_OVER_3333']/df['TSD_POP_EMP_TOTAL']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_WAGE_UNDER_3333_PCT \
                    + TSD_POP_EMP_WAGE_OVER_3333_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6449
Estimator:                   PanelOLS   R-squared (Between):              0.9110
No. Observations:                4002   R-squared (Within):               0.6449
Date:                Tue, May 05 2020   R-squared (Overall):              0.9067
Time:                        10:26:21   Log-likelihood                    1350.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      564.10
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          4.659e+04
                            

In [372]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6439
Estimator:                   PanelOLS   R-squared (Between):              0.9458
No. Observations:                4002   R-squared (Within):               0.6439
Date:                Tue, May 05 2020   R-squared (Overall):              0.9462
Time:                        01:28:33   Log-likelihood                    1345.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      561.87
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             561.87
                            

In [323]:
#df['TSD_POP_WAGE_UNDER_1250']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']
#df['TSD_WA_EMP_WAGE_UNDER_1250']=100*df['JOBS_LESS_THAN_1250_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TRACTS']
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #Looks the best one, but JTW goes the other way. Although JTW_HOME is still insignificant.

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6454
Estimator:                   PanelOLS   R-squared (Between):              0.9475
No. Observations:                4002   R-squared (Within):               0.6454
Date:                Mon, May 04 2020   R-squared (Overall):              0.9479
Time:                        14:57:27   Log-likelihood                    1353.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      565.39
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             565.39
                            

In [455]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_2018',
 'HH_MEAN_INC_2018',
 'GAS_PRICE_2018',

In [456]:
df['TSD_POP_WAGE_OVER_1250']=df['TOTAL_JOBS_RA_TRACTS']-df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']
df['TSD_POP_WAGE_OVER_1250_PCT']=100*df['TSD_POP_WAGE_OVER_1250']/df['TOTAL_JOBS_RA_TRACTS']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6454
Estimator:                   PanelOLS   R-squared (Between):              0.9088
No. Observations:                4002   R-squared (Within):               0.6454
Date:                Tue, May 05 2020   R-squared (Overall):              0.9049
Time:                        10:32:09   Log-likelihood                    1353.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      565.39
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          7.347e+04
                            

In [457]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #We can use this one as the base model

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6453
Estimator:                   PanelOLS   R-squared (Between):              0.9085
No. Observations:                4010   R-squared (Within):               0.6453
Date:                Tue, May 05 2020   R-squared (Overall):              0.9049
Time:                        10:34:11   Log-likelihood                    1358.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      618.17
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.305   Distribution:                 F(11,3737)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             774.24
                            

## Years since TNC arrived

In [458]:
#base model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6453
Estimator:                   PanelOLS   R-squared (Between):              0.9085
No. Observations:                4010   R-squared (Within):               0.6453
Date:                Tue, May 05 2020   R-squared (Overall):              0.9049
Time:                        10:34:31   Log-likelihood                    1358.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      618.17
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.305   Distribution:                 F(11,3737)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             774.24
                            

In [459]:
#TNC years after 2012 -- A little better

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_2012_BUS \
                    + YEARS_SINCE_TNC_2012_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #new base model

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6457
Estimator:                   PanelOLS   R-squared (Between):              0.9099
No. Observations:                4002   R-squared (Within):               0.6457
Date:                Tue, May 05 2020   R-squared (Overall):              0.9059
Time:                        10:34:47   Log-likelihood                    1354.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      566.07
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             709.14
                            

In [313]:
# exponentiated years since TNC

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + np.exp(YEARS_SINCE_TNC_BUS)\
                    + np.exp(YEARS_SINCE_TNC_RAIL) \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6438
Estimator:                   PanelOLS   R-squared (Between):              0.9295
No. Observations:                4002   R-squared (Within):               0.6438
Date:                Mon, May 04 2020   R-squared (Overall):              0.9301
Time:                        06:55:03   Log-likelihood                    1344.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      561.40
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             561.40
                            

# Uber Bookings

In [460]:
# base model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_2012_BUS \
                    + YEARS_SINCE_TNC_2012_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6457
Estimator:                   PanelOLS   R-squared (Between):              0.9099
No. Observations:                4002   R-squared (Within):               0.6457
Date:                Tue, May 05 2020   R-squared (Overall):              0.9059
Time:                        10:35:28   Log-likelihood                    1354.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      566.07
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             709.14
                            

In [461]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_BOOKINGS_log\
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6436
Estimator:                   PanelOLS   R-squared (Between):              0.9012
No. Observations:                4002   R-squared (Within):               0.6436
Date:                Tue, May 05 2020   R-squared (Overall):              0.8975
Time:                        10:35:39   Log-likelihood                    1342.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      612.05
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(11,3729)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             767.25
                            

In [462]:
#TNC bookings separated by bus and rail mode
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_BOOKINGS_BUS_log\
                    + TNC_BOOKINGS_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6437
Estimator:                   PanelOLS   R-squared (Between):              0.9021
No. Observations:                4002   R-squared (Within):               0.6437
Date:                Tue, May 05 2020   R-squared (Overall):              0.8983
Time:                        10:35:51   Log-likelihood                    1343.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      561.18
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          7.312e+04
                            

In [463]:
#separate Bookings by different clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_BOOKINGS_BUS_HINY_log \
                    + TNC_BOOKINGS_BUS_MIDLOW_log \
                    + TNC_BOOKINGS_RAIL_HINY_log \
                    + TNC_BOOKINGS_RAIL_MIDLOW_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #worse than Years Since TNC after 2012 variable

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6438
Estimator:                   PanelOLS   R-squared (Between):              0.9034
No. Observations:                4002   R-squared (Within):               0.6438
Date:                Tue, May 05 2020   R-squared (Overall):              0.8996
Time:                        10:36:01   Log-likelihood                    1344.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      481.04
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(14,3726)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          6.266e+04
                            

# Uber trips

In [464]:
# new base model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_2012_BUS \
                    + YEARS_SINCE_TNC_2012_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6457
Estimator:                   PanelOLS   R-squared (Between):              0.9099
No. Observations:                4002   R-squared (Within):               0.6457
Date:                Tue, May 05 2020   R-squared (Overall):              0.9059
Time:                        10:36:36   Log-likelihood                    1354.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      566.07
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             709.14
                            

In [465]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_BUS_log \
                    + TNC_TRIPS_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6437
Estimator:                   PanelOLS   R-squared (Between):              0.9023
No. Observations:                4002   R-squared (Within):               0.6437
Date:                Tue, May 05 2020   R-squared (Overall):              0.8986
Time:                        10:36:48   Log-likelihood                    1343.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      561.21
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          7.312e+04
                            

In [466]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_BUS_HINY_log \
                    + TNC_TRIPS_RAIL_HINY_log \
                    + TNC_TRIPS_BUS_MIDLOW_log \
                    + TNC_TRIPS_RAIL_MIDLOW_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6438
Estimator:                   PanelOLS   R-squared (Between):              0.9037
No. Observations:                4002   R-squared (Within):               0.6438
Date:                Tue, May 05 2020   R-squared (Overall):              0.9000
Time:                        10:37:00   Log-likelihood                    1344.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      481.09
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(14,3726)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          6.266e+04
                            

In [467]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_BUS_HINY_log \
                    + TNC_TRIPS_BUS_MID_OPEX_log \
                    + TNC_TRIPS_BUS_LOW_OPEX_log \
                    + TNC_TRIPS_RAIL_HINY_log \
                    + TNC_TRIPS_RAIL_MIDLOW_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ', data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6448
Estimator:                   PanelOLS   R-squared (Between):              0.9019
No. Observations:                4002   R-squared (Within):               0.6448
Date:                Tue, May 05 2020   R-squared (Overall):              0.8982
Time:                        10:37:19   Log-likelihood                    1350.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      450.87
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(15,3725)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          5.864e+04
                            

In [325]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_2018',
 'HH_MEAN_INC_2018',
 'GAS_PRICE_2018',

### Uber Scaled Revenue

In [379]:
# switch to Uber scaled revenue
# mod=PanelOLS.from_formula('UPT_ADJ_log \
#                     ~ VRM_ADJ_log \
#                     + FARE_per_UPT_2018_log \
#                     + POP_EMP_log \
#                     + GAS_PRICE_2018_log \
#                     + TOTAL_MED_INC_INDIV_2018_log \
#                     + PCT_HH_NO_VEH \
#                     + TSD_POP_PCT_LEHD \
#                     + JTW_HOME_PCT \
#                     + UBER_REV_SCALED_BUS \
#                     + UBER_REV_SCALED_RAIL \
#                     + BIKE_SHARE \
#                     + scooter_flag  \
#                     + EntityEffects \
#                     ',data=df)
# res=mod.fit()
# print(res)

In [52]:
# switch to Uber scaled revenue
# mod=PanelOLS.from_formula('UPT_ADJ_log \
#                     ~ VRM_ADJ_log \
#                     + FARE_per_UPT_2018_log \
#                     + POP_EMP_log \
#                     + GAS_PRICE_2018_log \
#                     + TOTAL_MED_INC_INDIV_2018_log \
#                     + PCT_HH_NO_VEH \
#                     + TSD_POP_PCT_LEHD \
#                     + JTW_HOME_PCT \
#                     + UBER_REV_SCALED_BUS_HINY \
#                     + UBER_REV_SCALED_BUS_MIDLOW \
#                     + UBER_REV_SCALED_RAIL_HINY \
#                     + UBER_REV_SCALED_RAIL_MIDLOW \
#                     + BIKE_SHARE \
#                     + scooter_flag  \
#                     + EntityEffects \
#                     ',data=df)
# res=mod.fit()
# print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6455
Estimator:                   PanelOLS   R-squared (Between):              0.9166
No. Observations:                3985   R-squared (Within):               0.6455
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9173
Time:                        15:37:01   Log-likelihood                    1342.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      482.54
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.268   Distribution:                 F(14,3710)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             482.54
                            

### Uber un-scaled revenue

In [468]:
# new base model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_2012_BUS \
                    + YEARS_SINCE_TNC_2012_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6457
Estimator:                   PanelOLS   R-squared (Between):              0.9099
No. Observations:                4002   R-squared (Within):               0.6457
Date:                Tue, May 05 2020   R-squared (Overall):              0.9059
Time:                        10:37:41   Log-likelihood                    1354.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      566.07
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             709.14
                            

In [469]:
# switch to Uber un-scaled revenue
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS \
                    + TNC_REVENUE_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6440
Estimator:                   PanelOLS   R-squared (Between):              0.9032
No. Observations:                4002   R-squared (Within):               0.6440
Date:                Tue, May 05 2020   R-squared (Overall):              0.8995
Time:                        10:37:53   Log-likelihood                    1345.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      561.88
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          7.317e+04
                            

In [328]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_2018',
 'HH_MEAN_INC_2018',
 'GAS_PRICE_2018',

In [470]:
# switch to Uber un-scaled revenue
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY \
                    + TNC_REVENUE_BUS_MIDLOW \
                    + TNC_REVENUE_RAIL_HINY \
                    + TNC_REVENUE_RAIL_MIDLOW \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6481
Estimator:                   PanelOLS   R-squared (Between):              0.9085
No. Observations:                4002   R-squared (Within):               0.6481
Date:                Tue, May 05 2020   R-squared (Overall):              0.9046
Time:                        10:38:06   Log-likelihood                    1368.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      490.15
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(14,3726)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          6.343e+04
                            

In [471]:
# switch to Uber un-scaled revenue
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY \
                    + TNC_REVENUE_BUS_MID_OPEX \
                    + TNC_REVENUE_BUS_LOW_OPEX \
                    + TNC_REVENUE_RAIL_HINY \
                    + TNC_REVENUE_RAIL_MID_OPEX \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #Revenue by clusters performs better

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6483
Estimator:                   PanelOLS   R-squared (Between):              0.9106
No. Observations:                4002   R-squared (Within):               0.6483
Date:                Tue, May 05 2020   R-squared (Overall):              0.9067
Time:                        10:38:32   Log-likelihood                    1369.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      457.81
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(15,3725)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             573.04
                            

In [472]:
# Use the logged transformed revenue

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6449
Estimator:                   PanelOLS   R-squared (Between):              0.9024
No. Observations:                4002   R-squared (Within):               0.6449
Date:                Tue, May 05 2020   R-squared (Overall):              0.8987
Time:                        10:38:49   Log-likelihood                    1350.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      450.91
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(15,3725)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          5.864e+04
                            

In [473]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_log \
                    + TNC_REVENUE_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6437
Estimator:                   PanelOLS   R-squared (Between):              0.9027
No. Observations:                4002   R-squared (Within):               0.6437
Date:                Tue, May 05 2020   R-squared (Overall):              0.8990
Time:                        10:41:37   Log-likelihood                    1343.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      561.26
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          7.312e+04
                            

In [474]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HI_OPEX_log \
                    + TNC_REVENUE_RAIL_HI_OPEX_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + TNC_REVENUE_BUS_NEW_YORK_log \
                    + TNC_REVENUE_RAIL_NEW_YORK_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6453
Estimator:                   PanelOLS   R-squared (Between):              0.9033
No. Observations:                4002   R-squared (Within):               0.6453
Date:                Tue, May 05 2020   R-squared (Overall):              0.8996
Time:                        10:42:00   Log-likelihood                    1352.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      398.38
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(17,3723)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          5.178e+04
                            

In [475]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6449
Estimator:                   PanelOLS   R-squared (Between):              0.9024
No. Observations:                4002   R-squared (Within):               0.6449
Date:                Tue, May 05 2020   R-squared (Overall):              0.8987
Time:                        10:42:06   Log-likelihood                    1350.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      450.91
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(15,3725)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          5.864e+04
                            

In [342]:
df['TNC_REVENUE_HINY']=df['TNC_REVENUE']*df['HI_OPEX']+df['TNC_REVENUE']*df['NEW_YORK']
df['TNC_REVENUE_HINY_log']=np.log(df['TNC_REVENUE_HINY']+1)

In [476]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #Performs about the same as TNC Trips
#new base model

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6449
Estimator:                   PanelOLS   R-squared (Between):              0.9024
No. Observations:                4002   R-squared (Within):               0.6449
Date:                Tue, May 05 2020   R-squared (Overall):              0.8987
Time:                        10:42:22   Log-likelihood                    1350.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      483.25
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(14,3726)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          6.285e+04
                            

In [337]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_2018',
 'HH_MEAN_INC_2018',
 'GAS_PRICE_2018',

# Per capita TNC trips

In [477]:
df['PER_CAPITA_TNC_TRIPS']=df['TNC_TRIPS']/df['Tot_Pop']

df['PER_CAPITA_TNC_TRIPS_HI_OPEX']=df['PER_CAPITA_TNC_TRIPS']*df['HI_OPEX']
df['PER_CAPITA_TNC_TRIPS_MID_OPEX']=df['PER_CAPITA_TNC_TRIPS']*df['MID_OPEX']
df['PER_CAPITA_TNC_TRIPS_LOW_OPEX']=df['PER_CAPITA_TNC_TRIPS']*df['LOW_OPEX']
df['PER_CAPITA_TNC_TRIPS_NEW_YORK']=df['PER_CAPITA_TNC_TRIPS']*df['NEW_YORK']
df['PER_CAPITA_TNC_TRIPS_HINY']=df['PER_CAPITA_TNC_TRIPS_HI_OPEX']+df['PER_CAPITA_TNC_TRIPS_NEW_YORK']
df['PER_CAPITA_TNC_TRIPS_MIDLOW']=df['PER_CAPITA_TNC_TRIPS_MID_OPEX']+df['PER_CAPITA_TNC_TRIPS_LOW_OPEX']

df['PER_CAPITA_TNC_TRIPS_HINY_BUS']=df['PER_CAPITA_TNC_TRIPS_HINY']*df['BUS_FLAG']
df['PER_CAPITA_TNC_TRIPS_HINY_RAIL']=df['PER_CAPITA_TNC_TRIPS_HINY']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_TRIPS_MIDLOW_BUS']=df['PER_CAPITA_TNC_TRIPS_MIDLOW']*df['BUS_FLAG']
df['PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL']=df['PER_CAPITA_TNC_TRIPS_MIDLOW']*df['RAIL_FLAG']

df['PER_CAPITA_TNC_BOOKINGS']=df['TNC_BOOKINGS']/df['Tot_Pop']
df['PER_CAPITA_TNC_BOOKINGS_HI_OPEX']=df['PER_CAPITA_TNC_BOOKINGS']*df['HI_OPEX']
df['PER_CAPITA_TNC_BOOKINGS_MID_OPEX']=df['PER_CAPITA_TNC_BOOKINGS']*df['MID_OPEX']
df['PER_CAPITA_TNC_BOOKINGS_LOW_OPEX']=df['PER_CAPITA_TNC_BOOKINGS']*df['LOW_OPEX']
df['PER_CAPITA_TNC_BOOKINGS_NEW_YORK']=df['PER_CAPITA_TNC_BOOKINGS']*df['NEW_YORK']
df['PER_CAPITA_TNC_BOOKINGS_HINY']=df['PER_CAPITA_TNC_BOOKINGS_HI_OPEX']+df['PER_CAPITA_TNC_BOOKINGS_NEW_YORK']
df['PER_CAPITA_TNC_BOOKINGS_MIDLOW']=df['PER_CAPITA_TNC_BOOKINGS_MID_OPEX']+df['PER_CAPITA_TNC_BOOKINGS_LOW_OPEX']

df['PER_CAPITA_TNC_BOOKINGS_HINY_BUS']=df['PER_CAPITA_TNC_BOOKINGS_HINY']*df['BUS_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_HINY_RAIL']=df['PER_CAPITA_TNC_BOOKINGS_HINY']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_MIDLOW_BUS']=df['PER_CAPITA_TNC_BOOKINGS_MIDLOW']*df['BUS_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_MIDLOW_RAIL']=df['PER_CAPITA_TNC_BOOKINGS_MIDLOW']*df['RAIL_FLAG']


In [478]:
df['PER_CAPITA_TNC_TRIPS_BUS']=df['PER_CAPITA_TNC_TRIPS']*df['BUS_FLAG']
df['PER_CAPITA_TNC_TRIPS_RAIL']=df['PER_CAPITA_TNC_TRIPS']*df['RAIL_FLAG']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_BUS \
                    + PER_CAPITA_TNC_TRIPS_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6444
Estimator:                   PanelOLS   R-squared (Between):              0.9032
No. Observations:                4002   R-squared (Within):               0.6444
Date:                Tue, May 05 2020   R-squared (Overall):              0.8995
Time:                        10:50:00   Log-likelihood                    1347.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      562.94
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          7.326e+04
                            

In [480]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6460
Estimator:                   PanelOLS   R-squared (Between):              0.9111
No. Observations:                4002   R-squared (Within):               0.6460
Date:                Tue, May 05 2020   R-squared (Overall):              0.9072
Time:                        10:52:41   Log-likelihood                    1356.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      485.69
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(14,3726)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          6.305e+04
                            

In [489]:
df['PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL']=df['PER_CAPITA_TNC_TRIPS_HI_OPEX']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS']=df['PER_CAPITA_TNC_TRIPS_HI_OPEX']*df['BUS_FLAG']
df['PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS']=df['PER_CAPITA_TNC_TRIPS_NEW_YORK']*df['BUS_FLAG']
df['PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL']=df['PER_CAPITA_TNC_TRIPS_NEW_YORK']*df['RAIL_FLAG']
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6474
Estimator:                   PanelOLS   R-squared (Between):              0.9104
No. Observations:                4002   R-squared (Within):               0.6474
Date:                Tue, May 05 2020   R-squared (Overall):              0.9065
Time:                        11:04:27   Log-likelihood                    1364.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      402.09
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(17,3723)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          5.209e+04
                            

In [490]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6369
Estimator:                   PanelOLS   R-squared (Between):              0.9367
No. Observations:                4124   R-squared (Within):               0.6369
Date:                Tue, May 05 2020   R-squared (Overall):              0.9371
Time:                        11:05:34   Log-likelihood                    1331.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      420.61
Entities:                         271   P-value                           0.0000
Avg Obs:                       15.218   Distribution:                 F(16,3837)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             420.61
                            

In [491]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6464
Estimator:                   PanelOLS   R-squared (Between):              0.9347
No. Observations:                4002   R-squared (Within):               0.6464
Date:                Tue, May 05 2020   R-squared (Overall):              0.9352
Time:                        11:06:27   Log-likelihood                    1358.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      400.27
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(17,3723)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             400.27
                            

In [492]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6468
Estimator:                   PanelOLS   R-squared (Between):              0.9422
No. Observations:                4002   R-squared (Within):               0.6468
Date:                Tue, May 05 2020   R-squared (Overall):              0.9427
Time:                        11:07:42   Log-likelihood                    1361.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      400.99
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(17,3723)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             400.99
                            

In [493]:
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']=df['TNC_TRIPS']/df['UPT_ADJ']
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS']=df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']*df['BUS_FLAG']
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL']=df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']*df['RAIL_FLAG']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6441
Estimator:                   PanelOLS   R-squared (Between):              0.9275
No. Observations:                4002   R-squared (Within):               0.6441
Date:                Tue, May 05 2020   R-squared (Overall):              0.9281
Time:                        11:11:14   Log-likelihood                    1346.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      562.27
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(12,3728)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             562.27
                            

In [494]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #USE THIS ONE

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6464
Estimator:                   PanelOLS   R-squared (Between):              0.9347
No. Observations:                4002   R-squared (Within):               0.6464
Date:                Tue, May 05 2020   R-squared (Overall):              0.9352
Time:                        11:13:15   Log-likelihood                    1358.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      400.27
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(17,3723)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             400.27
                            

In [497]:
df['PER_CAPITA_TNC_BOOKINGS_HI_OPEX_RAIL']=df['PER_CAPITA_TNC_BOOKINGS_HI_OPEX']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_HI_OPEX_BUS']=df['PER_CAPITA_TNC_BOOKINGS_HI_OPEX']*df['BUS_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_NEW_YORK_BUS']=df['PER_CAPITA_TNC_BOOKINGS_NEW_YORK']*df['BUS_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_NEW_YORK_RAIL']=df['PER_CAPITA_TNC_BOOKINGS_NEW_YORK']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_MID_OPEX_BUS']=df['PER_CAPITA_TNC_BOOKINGS_MID_OPEX']*df['BUS_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_MID_OPEX_RAIL']=df['PER_CAPITA_TNC_BOOKINGS_MID_OPEX']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_LOW_OPEX_BUS']=df['PER_CAPITA_TNC_BOOKINGS_LOW_OPEX']*df['BUS_FLAG']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_BOOKINGS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_BOOKINGS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_BOOKINGS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_BOOKINGS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_BOOKINGS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_BOOKINGS_MID_OPEX_RAIL \
                    + PER_CAPITA_TNC_BOOKINGS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6463
Estimator:                   PanelOLS   R-squared (Between):              0.9344
No. Observations:                4002   R-squared (Within):               0.6463
Date:                Tue, May 05 2020   R-squared (Overall):              0.9349
Time:                        11:16:59   Log-likelihood                    1358.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      400.22
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(17,3723)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             400.22
                            

In [ ]:
df['PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS']=df['PER_CAPITA_TNC_TRIPS_MID_OPEX']*df['BUS_FLAG']
df['PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL']=df['PER_CAPITA_TNC_TRIPS_MID_OPEX']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS']=df['PER_CAPITA_TNC_TRIPS_LOW_OPEX']*df['BUS_FLAG']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

In [482]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_2018',
 'HH_MEAN_INC_2018',
 'GAS_PRICE_2018',

In [483]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_BUS_HINY \
                    + TNC_REVENUE_BUS_MID_OPEX \
                    + TNC_REVENUE_BUS_LOW_OPEX \
                    + TNC_REVENUE_RAIL_HINY \
                    + TNC_REVENUE_RAIL_MID_OPEX \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6473
Estimator:                   PanelOLS   R-squared (Between):              0.9247
No. Observations:                4010   R-squared (Within):               0.6473
Date:                Tue, May 05 2020   R-squared (Overall):              0.9252
Time:                        10:55:20   Log-likelihood                    1369.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      456.70
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.305   Distribution:                 F(15,3733)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             456.70
                            

In [ ]:
# New base model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #USE THIS ONE

#separate bus and rail

# TNC Cost vs Transit Cost

In [388]:
# Test transit fare revenue against TNC fare revenue

df['TRANSIT_FARE_VS_TNC_FARE']=df['TNC_REVENUE']/(df['FARE_ADJ_2018'])
df['TNC_COST_PER_TRIP']=df['TNC_REVENUE']/df['TNC_TRIPS']
df['TNC_COST_VS_TRANSIT_FARE']=df['TNC_COST_PER_TRIP']/(df['FARE_per_UPT_2018'])

In [386]:
#base model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6449
Estimator:                   PanelOLS   R-squared (Between):              0.9288
No. Observations:                4002   R-squared (Within):               0.6449
Date:                Tue, May 05 2020   R-squared (Overall):              0.9294
Time:                        03:47:59   Log-likelihood                    1350.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      483.25
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(14,3726)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             483.25
                            

In [389]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_COST_VS_TRANSIT_FARE \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

ValueError: exog does not have full column rank.

### Separate Rail vs bus models

In [498]:
#new base model, for both modes

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #USE THIS ONE

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6464
Estimator:                   PanelOLS   R-squared (Between):              0.9347
No. Observations:                4002   R-squared (Within):               0.6464
Date:                Tue, May 05 2020   R-squared (Overall):              0.9352
Time:                        11:19:23   Log-likelihood                    1358.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      400.27
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(17,3723)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             400.27
                            

In [502]:
df['PER_CAPITA_TNC_TRIPS_HIMIDNY_BUS']=df['PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS']+df['PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS']+df['PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS']


mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HIMIDNY_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6431
Estimator:                   PanelOLS   R-squared (Between):              0.9121
No. Observations:                4002   R-squared (Within):               0.6431
Date:                Tue, May 05 2020   R-squared (Overall):              0.9129
Time:                        11:30:50   Log-likelihood                    1340.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      516.68
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(13,3727)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             516.68
                            

In [503]:
#bus model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HIMIDNY_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6822
Estimator:                   PanelOLS   R-squared (Between):              0.9732
No. Observations:                3429   R-squared (Within):               0.6822
Date:                Tue, May 05 2020   R-squared (Overall):              0.9732
Time:                        11:31:30   Log-likelihood                    1464.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      571.63
Entities:                         221   P-value                           0.0000
Avg Obs:                       15.516   Distribution:                 F(12,3196)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             571.63
                            

In [504]:
#rail mode

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5866
Estimator:                   PanelOLS   R-squared (Between):              0.8005
No. Observations:                 573   R-squared (Within):               0.5866
Date:                Tue, May 05 2020   R-squared (Overall):              0.8019
Time:                        11:32:38   Log-likelihood                    54.259
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      67.194
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(11,521)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             67.194
                            

In [499]:
# separate bus model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6860
Estimator:                   PanelOLS   R-squared (Between):              0.9861
No. Observations:                3429   R-squared (Within):               0.6860
Date:                Tue, May 05 2020   R-squared (Overall):              0.9861
Time:                        11:19:56   Log-likelihood                    1485.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      498.52
Entities:                         221   P-value                           0.0000
Avg Obs:                       15.516   Distribution:                 F(14,3194)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             498.52
                            

In [500]:
#rail model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5873
Estimator:                   PanelOLS   R-squared (Between):              0.8205
No. Observations:                 573   R-squared (Within):               0.5873
Date:                Tue, May 05 2020   R-squared (Overall):              0.8215
Time:                        11:20:52   Log-likelihood                    54.769
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      56.810
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(13,519)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             56.810
                            

In [345]:
# separate bus model with logged transformed TNC revenue

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects ', 
                    data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6829
Estimator:                   PanelOLS   R-squared (Between):              0.9773
No. Observations:                3429   R-squared (Within):               0.6829
Date:                Mon, May 04 2020   R-squared (Overall):              0.9773
Time:                        15:59:58   Log-likelihood                    1468.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      529.38
Entities:                         221   P-value                           0.0000
Avg Obs:                       15.516   Distribution:                 F(13,3195)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             529.38
                            

In [384]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_BUS_MIDLOW_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects ', 
                    data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res) #separate coeff for mid and low clusters seems to be better

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6821
Estimator:                   PanelOLS   R-squared (Between):              0.9795
No. Observations:                3429   R-squared (Within):               0.6821
Date:                Tue, May 05 2020   R-squared (Overall):              0.9796
Time:                        02:25:57   Log-likelihood                    1464.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      571.41
Entities:                         221   P-value                           0.0000
Avg Obs:                       15.516   Distribution:                 F(12,3196)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             571.41
                            

In [397]:
#base model for bus
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects ', 
                    data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6829
Estimator:                   PanelOLS   R-squared (Between):              0.9773
No. Observations:                3429   R-squared (Within):               0.6829
Date:                Tue, May 05 2020   R-squared (Overall):              0.9773
Time:                        04:03:14   Log-likelihood                    1468.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      529.38
Entities:                         221   P-value                           0.0000
Avg Obs:                       15.516   Distribution:                 F(13,3195)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             529.38
                            

## Rail Model

In [346]:
# separate rail model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_RAIL_HINY \
                    + TNC_REVENUE_RAIL_MID_OPEX \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res) #TSD low wages concentration coefficient doesn't work

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5843
Estimator:                   PanelOLS   R-squared (Between):              0.8362
No. Observations:                 573   R-squared (Within):               0.5843
Date:                Mon, May 04 2020   R-squared (Overall):              0.8370
Time:                        16:00:46   Log-likelihood                    52.692
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      60.904
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(12,520)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             60.904
                            

In [347]:
# separate rail model with log transformed TNC revenue

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5821
Estimator:                   PanelOLS   R-squared (Between):              0.7607
No. Observations:                 573   R-squared (Within):               0.5821
Date:                Mon, May 04 2020   R-squared (Overall):              0.7629
Time:                        16:01:10   Log-likelihood                    51.210
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      60.366
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(12,520)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             60.366
                            

In [348]:
#TNC revenue together
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5821
Estimator:                   PanelOLS   R-squared (Between):              0.7624
No. Observations:                 573   R-squared (Within):               0.5821
Date:                Mon, May 04 2020   R-squared (Overall):              0.7646
Time:                        16:01:45   Log-likelihood                    51.195
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      65.975
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(11,521)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             65.975
                            

In [350]:
# For Rail centric MSAs, the concentration of low-wage population may not affect that much. 

# df['TSD_POP_EMP']=df['TOTAL_JOBS_WA_TSD_TRACTS']+df['TOTAL_JOBS_RA_TSD_TRACTS']
# df['TSD_POP_EMP_TOTAL']=df['TOTAL_JOBS_WA_TRACTS']+df['TOTAL_JOBS_RA_TRACTS']
# df['TSD_POP_EMP_PCT']=df['TSD_POP_EMP']/df['TSD_POP_EMP_TOTAL']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res) #greater r-squared, but TSD variable still goes the other way

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5846
Estimator:                   PanelOLS   R-squared (Between):              0.7849
No. Observations:                 573   R-squared (Within):               0.5846
Date:                Mon, May 04 2020   R-squared (Overall):              0.7864
Time:                        16:04:21   Log-likelihood                    52.886
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      60.975
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(12,520)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             60.975
                            

In [395]:
# Test transit fare revenue against TNC fare revenue

df['TNC_REVENUE_VS_TRANSIT_REVENUE']=df['TNC_REVENUE']/(df['FARE_ADJ_2018'])
df['TNC_COST_PER_TRIP']=df['TNC_REVENUE']/df['TNC_TRIPS']
df['TNC_COST_PER_TRIP']=df['TNC_COST_PER_TRIP'].fillna(0)
#df['TNC_COST_VS_TRANSIT_FARE']=df['TNC_COST_PER_TRIP']/(df['FARE_ADJ_2018']/df['UPT_ADJ'])
df['TNC_COST_VS_TRANSIT_FARE']=np.where(df['TNC_COST_PER_TRIP']==0,0,df['TNC_COST_PER_TRIP']/(df['FARE_ADJ_2018']/df['UPT_ADJ']))

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_VS_TRANSIT_REVENUE \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5851
Estimator:                   PanelOLS   R-squared (Between):              0.7482
No. Observations:                 571   R-squared (Within):               0.5851
Date:                Tue, May 05 2020   R-squared (Overall):              0.7504
Time:                        03:59:52   Log-likelihood                    53.626
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      56.076
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.927   Distribution:                  F(13,517)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             56.076
                            

In [401]:

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_COST_VS_TRANSIT_FARE \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5853
Estimator:                   PanelOLS   R-squared (Between):              0.8371
No. Observations:                 573   R-squared (Within):               0.5853
Date:                Tue, May 05 2020   R-squared (Overall):              0.8377
Time:                        04:38:36   Log-likelihood                    53.408
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      66.854
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(11,521)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             66.854
                            

In [398]:
#removed fare
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_COST_VS_TRANSIT_FARE \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4461
Estimator:                   PanelOLS   R-squared (Between):              0.9918
No. Observations:                 573   R-squared (Within):               0.4461
Date:                Tue, May 05 2020   R-squared (Overall):              0.9931
Time:                        04:10:30   Log-likelihood                   -29.531
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      42.040
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(10,522)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             42.040
                            

In [400]:
#TSD variable going the other way
df['TNC_COST_PER_TRIP_log']=np.log(df['TNC_COST_PER_TRIP']+1)

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_COST_PER_TRIP_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5840
Estimator:                   PanelOLS   R-squared (Between):              0.7819
No. Observations:                 573   R-squared (Within):               0.5840
Date:                Tue, May 05 2020   R-squared (Overall):              0.7835
Time:                        04:36:20   Log-likelihood                    52.520
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      66.500
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(11,521)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             66.500
                            

In [399]:
df['TNC_COST_PER_TRIP_HI_OPEX']=df['TNC_COST_PER_TRIP']*df['HI_OPEX']
df['TNC_COST_PER_TRIP_MID_OPEX']=df['TNC_COST_PER_TRIP']*df['MID_OPEX']
df['TNC_COST_PER_TRIP_NY']=df['TNC_COST_PER_TRIP']*df['NEW_YORK']
df['TNC_COST_PER_TRIP_HINY']=df['TNC_COST_PER_TRIP_HI_OPEX']+df['TNC_COST_PER_TRIP_NY']

df['TNC_COST_PER_TRIP_HINY_log']=np.log(df['TNC_COST_PER_TRIP_HINY']+1)
df['TNC_COST_PER_TRIP_MID_OPEX_log']=np.log(df['TNC_COST_PER_TRIP_MID_OPEX']+1)

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_COST_PER_TRIP_HINY_log \
                    + TNC_COST_PER_TRIP_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5845
Estimator:                   PanelOLS   R-squared (Between):              0.7627
No. Observations:                 573   R-squared (Within):               0.5845
Date:                Tue, May 05 2020   R-squared (Overall):              0.7648
Time:                        04:36:13   Log-likelihood                    52.843
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      60.959
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(12,520)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             60.959
                            

In [403]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + TNC_COST_PER_TRIP_HINY_log \
                    + TNC_COST_PER_TRIP_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5840
Estimator:                   PanelOLS   R-squared (Between):              0.7740
No. Observations:                 573   R-squared (Within):               0.5840
Date:                Tue, May 05 2020   R-squared (Overall):              0.7758
Time:                        04:44:00   Log-likelihood                    52.516
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      60.840
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(12,520)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             60.840
                            

In [404]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_COST_PER_TRIP_HINY_log \
                    + TNC_COST_PER_TRIP_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5832
Estimator:                   PanelOLS   R-squared (Between):              0.6843
No. Observations:                 573   R-squared (Within):               0.5832
Date:                Tue, May 05 2020   R-squared (Overall):              0.6885
Time:                        04:46:38   Log-likelihood                    51.939
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      60.631
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(12,520)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             60.631
                            

In [405]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_WAGE_UNDER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_COST_PER_TRIP_HINY_log \
                    + TNC_COST_PER_TRIP_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5824
Estimator:                   PanelOLS   R-squared (Between):              0.7238
No. Observations:                 573   R-squared (Within):               0.5824
Date:                Tue, May 05 2020   R-squared (Overall):              0.7270
Time:                        04:47:49   Log-likelihood                    51.373
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      60.425
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(12,520)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             60.425
                            

In [406]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_RAIL_HINY \
                    + YEARS_SINCE_TNC_RAIL_MIDLOW \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5859
Estimator:                   PanelOLS   R-squared (Between):              0.7267
No. Observations:                 573   R-squared (Within):               0.5859
Date:                Tue, May 05 2020   R-squared (Overall):              0.7297
Time:                        04:49:54   Log-likelihood                    53.835
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      61.321
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(12,520)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             61.321
                            

In [407]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_RAIL_HINY \
                    + YEARS_SINCE_TNC_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5859
Estimator:                   PanelOLS   R-squared (Between):              0.7267
No. Observations:                 573   R-squared (Within):               0.5859
Date:                Tue, May 05 2020   R-squared (Overall):              0.7297
Time:                        04:50:39   Log-likelihood                    53.835
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      61.321
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(12,520)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             61.321
                            

In [349]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_2018',
 'HH_MEAN_INC_2018',
 'GAS_PRICE_2018',

# Look at the effect of years since 2012.
## Latest 7 time periods.

In [371]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['YEARS_AFTER_2010']>1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4133
Estimator:                   PanelOLS   R-squared (Between):              0.7228
No. Observations:                1800   R-squared (Within):               0.4133
Date:                Mon, May 04 2020   R-squared (Overall):              0.7233
Time:                        18:42:06   Log-likelihood                    1879.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      76.696
Entities:                         262   P-value                           0.0000
Avg Obs:                       6.8702   Distribution:                 F(14,1524)
Min Obs:                       1.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             76.696
                            

### Separate models by cluster

In [410]:
# combined model
#new base model, for both modes

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6449
Estimator:                   PanelOLS   R-squared (Between):              0.9288
No. Observations:                4002   R-squared (Within):               0.6449
Date:                Tue, May 05 2020   R-squared (Overall):              0.9294
Time:                        05:07:07   Log-likelihood                    1350.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      483.25
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(14,3726)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             483.25
                            

In [411]:
# high op-ex model

#new base model, for both modes

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_HINY_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[(df['HI_OPEX']==1) | (df['NEW_YORK']==1)])
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.8619
Estimator:                   PanelOLS   R-squared (Between):              0.1909
No. Observations:                 658   R-squared (Within):               0.8619
Date:                Tue, May 05 2020   R-squared (Overall):              0.1943
Time:                        05:08:33   Log-likelihood                    337.21
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      344.49
Entities:                          40   P-value                           0.0000
Avg Obs:                       16.450   Distribution:                  F(11,607)
Min Obs:                      10.0000                                           
Max Obs:                       17.000   F-statistic (robust):             344.49
                            

In [412]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[(df['HI_OPEX']==1) | (df['NEW_YORK']==1)])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.8654
Estimator:                   PanelOLS   R-squared (Between):              0.1304
No. Observations:                 658   R-squared (Within):               0.8654
Date:                Tue, May 05 2020   R-squared (Overall):              0.1341
Time:                        05:09:29   Log-likelihood                    345.52
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      324.61
Entities:                          40   P-value                           0.0000
Avg Obs:                       16.450   Distribution:                  F(12,606)
Min Obs:                      10.0000                                           
Max Obs:                       17.000   F-statistic (robust):             324.61
                            

In [414]:
# mid op-ex model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_BUS_log \
                    + TNC_REVENUE_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['MID_OPEX']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5885
Estimator:                   PanelOLS   R-squared (Between):              0.8724
No. Observations:                1322   R-squared (Within):               0.5885
Date:                Tue, May 05 2020   R-squared (Overall):              0.8713
Time:                        05:13:25   Log-likelihood                    522.79
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      159.29
Entities:                          86   P-value                           0.0000
Avg Obs:                       15.372   Distribution:                 F(11,1225)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             159.29
                            

In [415]:
# low op-ex model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_BUS_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['LOW_OPEX']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6366
Estimator:                   PanelOLS   R-squared (Between):              0.9716
No. Observations:                2030   R-squared (Within):               0.6366
Date:                Tue, May 05 2020   R-squared (Overall):              0.9715
Time:                        05:14:38   Log-likelihood                    786.14
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      330.07
Entities:                         136   P-value                           0.0000
Avg Obs:                       14.926   Distribution:                 F(10,1884)
Min Obs:                       6.0000                                           
Max Obs:                       17.000   F-statistic (robust):             330.07
                            

In [420]:
## excluding low op-ex, since they are different

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['LOW_OPEX']==0])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6678
Estimator:                   PanelOLS   R-squared (Between):              0.9121
No. Observations:                1980   R-squared (Within):               0.6678
Date:                Tue, May 05 2020   R-squared (Overall):              0.9131
Time:                        05:17:13   Log-likelihood                    626.08
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      308.59
Entities:                         126   P-value                           0.0000
Avg Obs:                       15.714   Distribution:                 F(12,1842)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             308.59
                            

In [425]:
# segment bus vs rail and low vs everything else
# rail

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5836
Estimator:                   PanelOLS   R-squared (Between):              0.8213
No. Observations:                 573   R-squared (Within):               0.5836
Date:                Tue, May 05 2020   R-squared (Overall):              0.8220
Time:                        05:30:10   Log-likelihood                    52.246
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      73.171
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(10,522)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             73.171
                            

In [432]:
# segment bus vs rail and low vs everything else
# bus -- mid and above

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[(df['RAIL_FLAG']==0) & df['LOW_OPEX']==0])
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6653
Estimator:                   PanelOLS   R-squared (Between):              0.8851
No. Observations:                1980   R-squared (Within):               0.6653
Date:                Tue, May 05 2020   R-squared (Overall):              0.8865
Time:                        05:32:38   Log-likelihood                    618.57
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      366.51
Entities:                         126   P-value                           0.0000
Avg Obs:                       15.714   Distribution:                 F(10,1844)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             366.51
                            

In [431]:
# segment bus vs rail and low vs everything else
# bus -- low op-ex

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[(df['RAIL_FLAG']==0) & df['LOW_OPEX']==1])
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6366
Estimator:                   PanelOLS   R-squared (Between):              0.9716
No. Observations:                2030   R-squared (Within):               0.6366
Date:                Tue, May 05 2020   R-squared (Overall):              0.9715
Time:                        05:31:56   Log-likelihood                    786.14
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      330.07
Entities:                         136   P-value                           0.0000
Avg Obs:                       14.926   Distribution:                 F(10,1884)
Min Obs:                       6.0000                                           
Max Obs:                       17.000   F-statistic (robust):             330.07
                            

### Currently prefer the combined model

In [433]:
# combined model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6449
Estimator:                   PanelOLS   R-squared (Between):              0.9288
No. Observations:                4002   R-squared (Within):               0.6449
Date:                Tue, May 05 2020   R-squared (Overall):              0.9294
Time:                        05:33:32   Log-likelihood                    1350.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      483.25
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(14,3726)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             483.25
                            

In [434]:
# remove JTW

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6448
Estimator:                   PanelOLS   R-squared (Between):              0.9267
No. Observations:                4010   R-squared (Within):               0.6448
Date:                Tue, May 05 2020   R-squared (Overall):              0.9273
Time:                        05:34:04   Log-likelihood                    1355.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      521.45
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.305   Distribution:                 F(13,3735)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             521.45
                            

In [435]:
# add maintenance
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6448
Estimator:                   PanelOLS   R-squared (Between):              0.9257
No. Observations:                4010   R-squared (Within):               0.6448
Date:                Tue, May 05 2020   R-squared (Overall):              0.9263
Time:                        05:34:52   Log-likelihood                    1355.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      484.13
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.305   Distribution:                 F(14,3734)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             484.13
                            

In [436]:
# maintenance variations
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total_log \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6438
Estimator:                   PanelOLS   R-squared (Between):              0.9068
No. Observations:                4010   R-squared (Within):               0.6438
Date:                Tue, May 05 2020   R-squared (Overall):              0.9076
Time:                        05:35:59   Log-likelihood                    1350.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      482.14
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.305   Distribution:                 F(14,3734)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             482.14
                            

In [437]:
# maintenance variations
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6438
Estimator:                   PanelOLS   R-squared (Between):              0.9047
No. Observations:                4010   R-squared (Within):               0.6438
Date:                Tue, May 05 2020   R-squared (Overall):              0.9055
Time:                        05:36:38   Log-likelihood                    1350.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      482.16
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.305   Distribution:                 F(14,3734)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             482.16
                            

In [438]:
# maintenance variations
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical_log \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6437
Estimator:                   PanelOLS   R-squared (Between):              0.9064
No. Observations:                4010   R-squared (Within):               0.6437
Date:                Tue, May 05 2020   R-squared (Overall):              0.9071
Time:                        05:36:44   Log-likelihood                    1349.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      481.90
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.305   Distribution:                 F(14,3734)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             481.90
                            

In [442]:
# combined

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6448
Estimator:                   PanelOLS   R-squared (Between):              0.9250
No. Observations:                4010   R-squared (Within):               0.6448
Date:                Tue, May 05 2020   R-squared (Overall):              0.9256
Time:                        05:38:15   Log-likelihood                    1355.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      484.20
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.305   Distribution:                 F(14,3734)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             484.20
                            

In [441]:
# slightly less segmentation
df['TNC_REVENUE_BUS_MIDHINY'] = df['TNC_REVENUE_BUS_MID_OPEX'] + df['TNC_REVENUE_BUS_HINY']

In [443]:
# current winner

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_BUS_MIDHINY \
                    + TNC_REVENUE_BUS_LOW_OPEX \
                    + TNC_REVENUE_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6446
Estimator:                   PanelOLS   R-squared (Between):              0.9387
No. Observations:                4010   R-squared (Within):               0.6446
Date:                Tue, May 05 2020   R-squared (Overall):              0.9392
Time:                        05:38:50   Log-likelihood                    1354.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      521.11
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.305   Distribution:                 F(13,3735)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             521.11
                            

In [445]:
#log transform
df['TNC_REVENUE_BUS_MIDHINY_log']=np.log(df['TNC_REVENUE_BUS_MIDHINY']+1)


mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6448
Estimator:                   PanelOLS   R-squared (Between):              0.9250
No. Observations:                4010   R-squared (Within):               0.6448
Date:                Tue, May 05 2020   R-squared (Overall):              0.9256
Time:                        05:42:10   Log-likelihood                    1355.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      484.20
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.305   Distribution:                 F(14,3734)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             484.20
                            

In [446]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)



                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6448
Estimator:                   PanelOLS   R-squared (Between):              0.9250
No. Observations:                4010   R-squared (Within):               0.6448
Date:                Tue, May 05 2020   R-squared (Overall):              0.9256
Time:                        05:42:50   Log-likelihood                    1355.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      451.80
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.305   Distribution:                 F(15,3733)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             451.80
                            

### Bus vs rail with just TNCs segmented by op-ex

In [448]:
# bus

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_BUS_MIDHINY_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6818
Estimator:                   PanelOLS   R-squared (Between):              0.9729
No. Observations:                3437   R-squared (Within):               0.6818
Date:                Tue, May 05 2020   R-squared (Overall):              0.9729
Time:                        05:44:18   Log-likelihood                    1468.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      624.28
Entities:                         221   P-value                           0.0000
Avg Obs:                       15.552   Distribution:                 F(11,3205)
Min Obs:                       6.0000                                           
Max Obs:                       17.000   F-statistic (robust):             624.28
                            

In [450]:
# rail

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5836
Estimator:                   PanelOLS   R-squared (Between):              0.8213
No. Observations:                 573   R-squared (Within):               0.5836
Date:                Tue, May 05 2020   R-squared (Overall):              0.8220
Time:                        05:45:07   Log-likelihood                    52.246
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      73.171
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(10,522)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             73.171
                            

### Current best models below, split bus vs rail

In [451]:
# bus - simplified

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_BUS_MIDHINY_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6818
Estimator:                   PanelOLS   R-squared (Between):              0.9729
No. Observations:                3437   R-squared (Within):               0.6818
Date:                Tue, May 05 2020   R-squared (Overall):              0.9729
Time:                        05:45:58   Log-likelihood                    1468.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      624.28
Entities:                         221   P-value                           0.0000
Avg Obs:                       15.552   Distribution:                 F(11,3205)
Min Obs:                       6.0000                                           
Max Obs:                       17.000   F-statistic (robust):             624.28
                            

In [452]:
# rail -- simplified

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5836
Estimator:                   PanelOLS   R-squared (Between):              0.8213
No. Observations:                 573   R-squared (Within):               0.5836
Date:                Tue, May 05 2020   R-squared (Overall):              0.8220
Time:                        05:46:20   Log-likelihood                    52.246
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      73.171
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(10,522)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             73.171
                            

# Models

## Combined Model

In [508]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HIMIDNY_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6431
Estimator:                   PanelOLS   R-squared (Between):              0.9121
No. Observations:                4002   R-squared (Within):               0.6431
Date:                Tue, May 05 2020   R-squared (Overall):              0.9129
Time:                        11:51:39   Log-likelihood                    1340.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      516.68
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.275   Distribution:                 F(13,3727)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             516.68
                            

In [521]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6860
Estimator:                   PanelOLS   R-squared (Between):              0.9861
No. Observations:                3429   R-squared (Within):               0.6860
Date:                Tue, May 05 2020   R-squared (Overall):              0.9861
Time:                        11:55:52   Log-likelihood                    1485.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      498.52
Entities:                         221   P-value                           0.0000
Avg Obs:                       15.516   Distribution:                 F(14,3194)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             498.52
                            

In [534]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5873
Estimator:                   PanelOLS   R-squared (Between):              0.8205
No. Observations:                 573   R-squared (Within):               0.5873
Date:                Tue, May 05 2020   R-squared (Overall):              0.8215
Time:                        12:00:41   Log-likelihood                    54.769
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      56.810
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(13,519)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             56.810
                            

In [535]:
# keep the relevant fields in the data set
app_data = pd.concat([df[['RAIL_FLAG', 'CLUSTER_APTA']], 
                      res.model.dependent.dataframe, 
                      res.model.exog.dataframe, 
                      res.estimated_effects, 
                      res.resids], axis=1)

In [536]:
# calculate the fitted values
app_data['fitted'] = app_data['estimated_effects']
for var in res.params.keys(): 
    app_data['fitted'] = app_data['fitted'] + (res.params[var] * app_data[var])

# and check that we add up correctly
app_data['check_fitted'] = app_data['fitted'] + app_data['residual'] - app_data['UPT_ADJ_log']

In [537]:
# create the necessary columns

# dependents
app_data['UPT_ADJ']    = np.exp(app_data['UPT_ADJ_log']) - 1
app_data['fitted_exp'] = np.exp(app_data['fitted']) - 1

# linear versions of logged variables
for var in res.params.keys(): 
    if '_log' in var: 
        app_data[var.replace('_log', '')] = np.exp(app_data[var]) - 1              

# ratios and differences
for var in res.params.keys(): 
    if '_log' in var: 
        app_data[var.replace('_log', '_ratio')] = 1
    else: 
        app_data[var+'_diff'] = 0

# FAC multipliers
for var in res.params.keys(): 
    app_data[var+'_FAC_ratio'] = 1

app_data['UPT_ADJ_base'] = 0
app_data['UPT_ADJ_ratio'] = 1
app_data['fitted_exp_ratio'] = 1
app_data['UPT_ADJ_diff'] = 0
app_data['fitted_exp_diff'] = 0

# for tracking new systems
app_data['UPT_ADJ_first_year'] = 0
app_data['UPT_ADJ_new_reporter'] = 0

In [538]:
# get the ridership in the first year the system starts. This allows us to track new systems. 
# calculate the FAC relative to a specific base year
ids = app_data.index.get_level_values(0).unique()

for id in ids: 
    years = app_data.loc[id].index.get_level_values(0).sort_values()
    first_year = years[0]

    app_data.loc[(id,first_year),'UPT_ADJ_new_reporter'] = app_data.loc[(id,first_year),'UPT_ADJ']
    
    for year in years: 
        app_data.loc[(id,year),'UPT_ADJ_first_year']   = app_data.loc[(id,first_year),'UPT_ADJ']

In [539]:
# calculate the FAC relative to a specific base year
ids = app_data.index.get_level_values(0).unique()

for id in ids: 
    years = app_data.loc[id].index.get_level_values(0).sort_values()

    for year in years[1:]:       
        base_year = year-1
        
        for var in res.params.keys(): 
            # ratios and differences
            if '_log' in var: 
                out_var = var.replace('_log', '_ratio')
                app_data.loc[(id,year), out_var] = (app_data.loc[(id,year), var] 
                                                 / app_data.loc[(id,base_year), var])
            else: 
                out_var = var+'_diff'                    
                app_data.loc[(id,year), out_var] = (app_data.loc[(id,year), var] 
                                                 - app_data.loc[(id,base_year), var])

            # FAC multipliers
            app_data.loc[(id,year),var+'_FAC_ratio'] = np.exp(res.params[var] * (
                                                    app_data.loc[(id,year), var] 
                                                    - app_data.loc[(id,base_year), var]))

            # estimated effects (if time effects is zero, no change)
            app_data.loc[(id,year),'effects_FAC_ratio'] = np.exp(
                                                    app_data.loc[(id,year), 'estimated_effects'] 
                                                    - app_data.loc[(id,base_year), 'estimated_effects'])

            # residual
            app_data.loc[(id,year),'residual_FAC_ratio'] = np.exp(
                                                    app_data.loc[(id,year), 'residual'] 
                                                    - app_data.loc[(id,base_year), 'residual'])

        # observed and fitted changes            
        app_data.loc[(id,year),'UPT_ADJ_base'] = app_data.loc[(id,base_year),'UPT_ADJ']
        app_data.loc[(id,year),'UPT_ADJ_ratio'] = (app_data.loc[(id,year),'UPT_ADJ'] 
                                                 / app_data.loc[(id,base_year),'UPT_ADJ'])
        app_data.loc[(id,year),'fitted_exp_ratio'] = (app_data.loc[(id,year),'fitted_exp'] 
                                                 / app_data.loc[(id,base_year),'fitted_exp'])

        app_data.loc[(id,year),'UPT_ADJ_diff'] = (app_data.loc[(id,year),'UPT_ADJ'] 
                                                 - app_data.loc[(id,base_year),'UPT_ADJ'])
        app_data.loc[(id,year),'fitted_exp_diff'] = (app_data.loc[(id,year),'fitted_exp'] 
                                                 - app_data.loc[(id,base_year),'fitted_exp'])

/Users/rianajawad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
/Users/rianajawad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app


In [540]:
# calculate the FAC
app_data['FAC_Sum'] = 0
for var in res.params.keys(): 
    app_data[var+'_FAC'] = (app_data[var+'_FAC_ratio'] - 1) * app_data['UPT_ADJ_base']
    app_data['FAC_Sum'] = app_data['FAC_Sum'] + app_data[var+'_FAC']




In [541]:

# total FAC is based on the fitted model (applied multiplicitively)
app_data['Known_FAC'] = (app_data['fitted_exp_ratio'] - 1) * app_data['UPT_ADJ_base']

# uknown change is the difference between the observed change and the known change
app_data['Unknown_FAC'] = app_data['UPT_ADJ_diff'] - app_data['Known_FAC']

# the change in ridership associated new systems
app_data['New_Reporter_FAC'] = app_data['UPT_ADJ_new_reporter'] 

# should be teh same as UPT_ADJ_DIFF
app_data['Total_Change'] = app_data['Known_FAC'] + app_data['Unknown_FAC'] + app_data['New_Reporter_FAC']

In [542]:
# calculate APTA - 4 groups
app_data['CLUSTER_APTA4'] = np.floor(app_data['CLUSTER_APTA']/10)

In [543]:
# reset the index so I get ID and year
app_data = app_data.reset_index()
app_data = app_data.rename(columns={'level_0' : 'ID', 'level_1' : 'Year'})
app_data.to_csv('FAC_RAIL_Model.csv')

In [544]:
# these are the fields we keep
out_fields = ['ID', 'Year', 'RAIL_FLAG', 
              'CLUSTER_APTA', 'CLUSTER_APTA4',    
              'UPT_ADJ_first_year', 
              'UPT_ADJ_base', 'UPT_ADJ', 'UPT_ADJ_diff', 
              'fitted_exp', 'fitted_exp_diff']

# exogenous variables 
for var in res.params.keys(): 
    out_var = var.replace('_log', '')
    out_fields = out_fields + [out_var]
    
# FAC
for var in res.params.keys(): 
    out_fields = out_fields + [var+'_FAC']
out_fields = out_fields + ['FAC_Sum', 'Known_FAC', 'Unknown_FAC', 'New_Reporter_FAC', 'Total_Change']
    
# keep those fields
summary_data = app_data.reset_index()[out_fields]

In [545]:
# take weighted average of exogenous variables
for var in res.params.keys(): 
    out_var = var.replace('_log', '')
    summary_data[out_var] = summary_data[out_var] * summary_data['UPT_ADJ_first_year']

# aggregate to bus/rail totals
summary_data_apta4 = summary_data.groupby(by=['CLUSTER_APTA4', 'RAIL_FLAG', 'Year']).agg('sum')

# divide for weighted averages
for var in res.params.keys(): 
    out_var = var.replace('_log', '')
    summary_data_apta4[out_var] = summary_data_apta4[out_var] / summary_data_apta4['UPT_ADJ_first_year']

In [546]:

summary_data_apta4.to_csv('FAC_totals_APTA4_CLUSTERS_Rail_Model.csv')


